In [1]:
import mss
import cv2
import numpy as np
import time
import pyautogui
import json
import os
import hashlib
import asyncio
import logging
from datetime import datetime
from typing import List, Dict, Tuple, Optional, Union
from dataclasses import dataclass, asdict
from pathlib import Path
from collections import deque
import threading
from concurrent.futures import ThreadPoolExecutor
import gc
import psutil
import warnings
warnings.filterwarnings('ignore')

# ML Libraries
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import joblib
import pickle

# Computer Vision
from skimage.feature import local_binary_pattern, hog
from skimage.measure import regionprops
from scipy import ndimage, stats
from scipy.spatial.distance import euclidean

# Security & Config
import configparser
import secrets
from cryptography.fernet import Fernet
import tempfile

# Advanced Statistics
try:
    from statsmodels.stats.proportion import proportion_confint
    from statsmodels.stats.contingency_tables import mcnemar
    HAS_STATSMODELS = True
    print("✅ statsmodels متاحة - سيتم استخدام إحصائيات متقدمة")
except ImportError:
    HAS_STATSMODELS = False
    print("⚠️ statsmodels غير مثبتة - سيتم استخدام إحصائيات بسيطة")

# ================================
# Advanced Statistics Helper
# ================================

def calculate_confidence_interval(scores, confidence=0.95):
    """حساب فترة الثقة للنتائج مع دعم statsmodels"""
    try:
        if len(scores) < 2:
            return np.mean(scores) if scores else 0, 0, 0
        
        mean_score = np.mean(scores)
        
        if HAS_STATSMODELS and len(scores) > 5:
            # استخدام statsmodels للحسابات المتقدمة
            std_err = stats.sem(scores)
            h = std_err * stats.t.ppf((1 + confidence) / 2., len(scores)-1)
            return mean_score, mean_score - h, mean_score + h
        else:
            # حساب بسيط
            std_err = stats.sem(scores)
            h = std_err * stats.t.ppf((1 + confidence) / 2., len(scores)-1)
            return mean_score, mean_score - h, mean_score + h
            
    except Exception as e:
        return np.mean(scores) if scores else 0, 0, 0

def calculate_accuracy_confidence_interval(y_true, y_pred, confidence=0.95):
    """حساب فترة الثقة لدقة النموذج"""
    try:
        if HAS_STATSMODELS:
            correct = np.sum(y_true == y_pred)
            total = len(y_true)
            ci_low, ci_upp = proportion_confint(correct, total, alpha=1-confidence)
            return accuracy_score(y_true, y_pred), ci_low, ci_upp
        else:
            accuracy = accuracy_score(y_true, y_pred)
            return accuracy, max(0, accuracy - 0.1), min(1, accuracy + 0.1)
    except:
        return 0, 0, 0

# ================================
# Configuration & Security Layer
# ================================

@dataclass
class SecurityConfig:
    """إعدادات الأمان المتقدمة"""
    max_memory_usage_mb: int = 1024
    max_cpu_usage_percent: float = 80.0
    encryption_enabled: bool = True
    secure_temp_dir: bool = True
    max_file_size_mb: int = 100
    allowed_extensions: List[str] = None
    
    def __post_init__(self):
        if self.allowed_extensions is None:
            self.allowed_extensions = ['.png', '.jpg', '.jpeg', '.json', '.pkl']

@dataclass
class PerformanceConfig:
    """إعدادات الأداء المحسنة"""
    frame_buffer_size: int = 3
    roi_update_interval: float = 1.0
    memory_cleanup_interval: float = 30.0
    max_screenshot_resolution: Tuple[int, int] = (1920, 1080)
    adaptive_quality: bool = True
    parallel_processing: bool = True
    max_workers: int = 4

@dataclass
class LearningConfig:
    """إعدادات التعلم التكيفي"""
    min_samples_for_learning: int = 10
    learning_rate: float = 0.1
    adaptation_speed: float = 0.05
    pattern_memory_size: int = 100
    success_weight: float = 1.0
    failure_weight: float = 0.5
    time_decay_factor: float = 0.95

class SecureConfigManager:
    """مدير الإعدادات الآمن"""
    
    def __init__(self, config_path: str = "config/captcha_config.ini"):
        self.config_path = Path(config_path)
        self.config_path.parent.mkdir(parents=True, exist_ok=True)
        self.config = configparser.ConfigParser()
        self.encryption_key = self._get_or_create_key()
        self.cipher = Fernet(self.encryption_key)
        self._load_config()
    
    def _get_or_create_key(self) -> bytes:
        """إنشاء أو تحميل مفتاح التشفير"""
        key_file = self.config_path.parent / ".key"
        try:
            if key_file.exists():
                return key_file.read_bytes()
            else:
                key = Fernet.generate_key()
                key_file.write_bytes(key)
                key_file.chmod(0o600)
                return key
        except Exception as e:
            logging.warning(f"فشل في إدارة مفتاح التشفير: {e}")
            return Fernet.generate_key()
    
    def _load_config(self):
        """تحميل الإعدادات"""
        try:
            if self.config_path.exists():
                self.config.read(self.config_path)
            else:
                self._create_default_config()
        except Exception as e:
            logging.error(f"خطأ في تحميل الإعدادات: {e}")
            self._create_default_config()
    
    def _create_default_config(self):
        """إنشاء إعدادات افتراضية"""
        self.config['PATHS'] = {
            'templates_dir': 'templates',
            'memory_dir': 'captcha_memory',
            'logs_dir': 'logs',
            'temp_dir': 'temp'
        }
        
        self.config['SECURITY'] = {
            'max_memory_mb': '1024',
            'max_cpu_percent': '80.0',
            'encryption_enabled': 'true',
            'secure_temp': 'true'
        }
        
        self.config['PERFORMANCE'] = {
            'frame_buffer_size': '3',
            'roi_update_interval': '1.0',
            'memory_cleanup_interval': '30.0',
            'parallel_processing': 'true',
            'max_workers': '4'
        }
        
        self.config['ML'] = {
            'model_retrain_interval': '600',
            'min_training_samples': '50',
            'feature_selection_k': '15',
            'cross_validation_folds': '5'
        }
        
        self.config['LEARNING'] = {
            'min_samples_for_learning': '10',
            'learning_rate': '0.1',
            'adaptation_speed': '0.05',
            'pattern_memory_size': '100',
            'auto_save_interval': '300'
        }
        
        self.save_config()
    
    def save_config(self):
        """حفظ الإعدادات"""
        try:
            with open(self.config_path, 'w') as f:
                self.config.write(f)
        except Exception as e:
            logging.error(f"خطأ في حفظ الإعدادات: {e}")
    
    def get_template_path(self, template_name: str) -> Path:
        """الحصول على مسار آمن للقالب"""
        templates_dir = Path(self.config.get('PATHS', 'templates_dir', fallback='templates'))
        templates_dir.mkdir(parents=True, exist_ok=True)
        
        safe_name = "".join(c for c in template_name if c.isalnum() or c in "._-")
        return templates_dir / f"{safe_name}.png"
    
    def get_secure_path(self, section: str, key: str, filename: str = "") -> Path:
        """الحصول على مسار آمن"""
        base_dir = Path(self.config.get('PATHS', key, fallback=key))
        base_dir.mkdir(parents=True, exist_ok=True)
        
        if filename:
            safe_filename = "".join(c for c in filename if c.isalnum() or c in "._-")
            return base_dir / safe_filename
        return base_dir

# ================================
# Advanced Performance Monitor
# ================================

class PerformanceMonitor:
    """مراقب الأداء المتقدم"""
    
    def __init__(self, config: PerformanceConfig):
        self.config = config
        self.process = psutil.Process()
        self.memory_history = deque(maxlen=100)
        self.cpu_history = deque(maxlen=100)
        self.last_cleanup = time.time()
        self.performance_alerts = []
        
    def check_resources(self) -> Dict[str, Union[bool, float]]:
        """فحص الموارد المتاحة"""
        try:
            memory_mb = self.process.memory_info().rss / 1024 / 1024
            cpu_percent = self.process.cpu_percent()
            
            self.memory_history.append(memory_mb)
            self.cpu_history.append(cpu_percent)
            
            return {
                'memory_ok': memory_mb < self.config.max_memory_usage_mb,
                'cpu_ok': cpu_percent < self.config.max_cpu_usage_percent,
                'memory_mb': memory_mb,
                'cpu_percent': cpu_percent,
                'memory_trend': self._calculate_trend(self.memory_history),
                'cpu_trend': self._calculate_trend(self.cpu_history)
            }
        except Exception as e:
            logging.warning(f"خطأ في فحص الموارد: {e}")
            return {'memory_ok': True, 'cpu_ok': True, 'memory_mb': 0, 'cpu_percent': 0}
    
    def _calculate_trend(self, history: deque) -> str:
        """حساب اتجاه الاستخدام"""
        if len(history) < 5:
            return "stable"
        
        recent = list(history)[-5:]
        older = list(history)[-10:-5] if len(history) >= 10 else recent
        
        recent_avg = np.mean(recent)
        older_avg = np.mean(older)
        
        if recent_avg > older_avg * 1.2:
            return "increasing"
        elif recent_avg < older_avg * 0.8:
            return "decreasing"
        return "stable"
    
    def should_cleanup(self) -> bool:
        """تحديد ما إذا كان يجب تنظيف الذاكرة"""
        current_time = time.time()
        if current_time - self.last_cleanup > self.config.memory_cleanup_interval:
            self.last_cleanup = current_time
            return True
        
        resources = self.check_resources()
        return not resources['memory_ok'] or resources['memory_trend'] == 'increasing'
    
    def cleanup_memory(self):
        """تنظيف الذاكرة"""
        try:
            gc.collect()
            logging.info("تم تنظيف الذاكرة")
        except Exception as e:
            logging.warning(f"خطأ في تنظيف الذاكرة: {e}")

# ================================
# Adaptive Learning Engine
# ================================

class AdaptiveLearningEngine:
    """محرك التعلم التكيفي المتقدم"""
    
    def __init__(self, config: LearningConfig, config_manager: SecureConfigManager):
        self.config = config
        self.config_manager = config_manager
        
        # بيانات التعلم
        self.timing_patterns = {
            'verified_wait': deque(maxlen=config.pattern_memory_size),
            'success_wait': deque(maxlen=config.pattern_memory_size),
            'click_delays': deque(maxlen=config.pattern_memory_size)
        }
        
        self.success_patterns = {
            'click_positions': deque(maxlen=config.pattern_memory_size),
            'timing_sequences': deque(maxlen=config.pattern_memory_size),
            'context_patterns': deque(maxlen=config.pattern_memory_size)
        }
        
        self.adaptive_parameters = {
            'verified_timeout': 15.0,
            'success_timeout': 20.0,
            'click_delay': 0.3,
            'check_interval': 0.2,
            'confidence_threshold': 0.7
        }
        
        self.learning_stats = {
            'total_samples': 0,
            'successful_adaptations': 0,
            'failed_adaptations': 0,
            'last_adaptation': None,
            'adaptation_history': deque(maxlen=50)
        }
        
        self._load_learning_data()
    
    def _load_learning_data(self):
        """تحميل بيانات التعلم المحفوظة"""
        try:
            learning_file = self.config_manager.get_secure_path('PATHS', 'memory_dir', 'adaptive_learning.json')
            
            if learning_file.exists():
                with open(learning_file, 'r', encoding='utf-8') as f:
                    saved_data = json.load(f)
                
                # استعادة المعاملات التكيفية
                if 'adaptive_parameters' in saved_data:
                    self.adaptive_parameters.update(saved_data['adaptive_parameters'])
                
                # استعادة الإحصائيات
                if 'learning_stats' in saved_data:
                    self.learning_stats.update(saved_data['learning_stats'])
                    # تحويل التاريخ إلى deque
                    if 'adaptation_history' in saved_data['learning_stats']:
                        self.learning_stats['adaptation_history'] = deque(
                            saved_data['learning_stats']['adaptation_history'],
                            maxlen=50
                        )
                
                # استعادة الأنماط (الأحدث فقط)
                if 'timing_patterns' in saved_data:
                    for key, values in saved_data['timing_patterns'].items():
                        if key in self.timing_patterns:
                         self.timing_patterns[key] = deque(values[-self.config.pattern_memory_size:], 
                         maxlen=self.config.pattern_memory_size)
                
                if 'success_patterns' in saved_data:
                    for key, values in saved_data['success_patterns'].items():
                        if key in self.success_patterns:
                            self.success_patterns[key] = deque(values[-self.config.pattern_memory_size:], 
                                                             maxlen=self.config.pattern_memory_size)
                
                logging.info(f"تم تحميل بيانات التعلم: {self.learning_stats['total_samples']} عينة")
                
        except Exception as e:
            logging.warning(f"خطأ في تحميل بيانات التعلم: {e}")
    
    def _save_learning_data(self):
        """حفظ بيانات التعلم"""
        try:
            learning_file = self.config_manager.get_secure_path('PATHS', 'memory_dir', 'adaptive_learning.json')
            
            save_data = {
                'adaptive_parameters': self.adaptive_parameters.copy(),
                'learning_stats': {
                    'total_samples': self.learning_stats['total_samples'],
                    'successful_adaptations': self.learning_stats['successful_adaptations'],
                    'failed_adaptations': self.learning_stats['failed_adaptations'],
                    'last_adaptation': self.learning_stats['last_adaptation'],
                    'adaptation_history': list(self.learning_stats['adaptation_history'])
                },
                'timing_patterns': {
                    key: list(values) for key, values in self.timing_patterns.items()
                },
                'success_patterns': {
                    key: list(values) for key, values in self.success_patterns.items()
                },
                'save_timestamp': datetime.now().isoformat()
            }
            
            with open(learning_file, 'w', encoding='utf-8') as f:
                json.dump(save_data, f, indent=2, ensure_ascii=False)
                
        except Exception as e:
            logging.warning(f"خطأ في حفظ بيانات التعلم: {e}")
    
    def learn_from_timing(self, event_type: str, duration: float, success: bool, context: Dict = None):
        """التعلم من أوقات الأحداث"""
        try:
            current_time = time.time()
            
            # تسجيل التوقيت
            timing_record = {
                'duration': duration,
                'success': success,
                'timestamp': current_time,
                'context': context or {}
            }
            
            if event_type in self.timing_patterns:
                self.timing_patterns[event_type].append(timing_record)
            
            # تحديث المعاملات التكيفية
            if success:
                self._adapt_timing_parameters(event_type, duration, True)
                self.learning_stats['successful_adaptations'] += 1
            else:
                self._adapt_timing_parameters(event_type, duration, False)
                self.learning_stats['failed_adaptations'] += 1
            
            self.learning_stats['total_samples'] += 1
            self.learning_stats['last_adaptation'] = current_time
            
            # حفظ تاريخ التكيف
            adaptation_record = {
                'timestamp': current_time,
                'event_type': event_type,
                'duration': duration,
                'success': success,
                'parameters_before': self.adaptive_parameters.copy()
            }
            
            self.learning_stats['adaptation_history'].append(adaptation_record)
            
            logging.debug(f"تعلم من {event_type}: {duration:.2f}s، نجاح: {success}")
            
        except Exception as e:
            logging.warning(f"خطأ في التعلم من التوقيت: {e}")
    
    def _adapt_timing_parameters(self, event_type: str, duration: float, success: bool):
        """تكييف معاملات التوقيت بناءً على النتائج"""
        try:
            adaptation_rate = self.config.adaptation_speed
            
            if event_type == 'verified_wait':
                current_timeout = self.adaptive_parameters['verified_timeout']
                
                if success:
                    # إذا نجح في وقت أقل من المهلة، قلل المهلة تدريجياً
                    if duration < current_timeout * 0.8:
                        new_timeout = current_timeout * (1 - adaptation_rate)
                        self.adaptive_parameters['verified_timeout'] = max(5.0, new_timeout)
                else:
                    # إذا فشل، زد المهلة
                    new_timeout = current_timeout * (1 + adaptation_rate)
                    self.adaptive_parameters['verified_timeout'] = min(30.0, new_timeout)
            
            elif event_type == 'success_wait':
                current_timeout = self.adaptive_parameters['success_timeout']
                
                if success:
                    if duration < current_timeout * 0.8:
                        new_timeout = current_timeout * (1 - adaptation_rate)
                        self.adaptive_parameters['success_timeout'] = max(8.0, new_timeout)
                else:
                    new_timeout = current_timeout * (1 + adaptation_rate)
                    self.adaptive_parameters['success_timeout'] = min(40.0, new_timeout)
            
            elif event_type == 'click_delays':
                current_delay = self.adaptive_parameters['click_delay']
                
                if success:
                    # تسريع النقر تدريجياً عند النجاح
                    new_delay = current_delay * (1 - adaptation_rate * 0.5)
                    self.adaptive_parameters['click_delay'] = max(0.1, new_delay)
                else:
                    # إبطاء النقر عند الفشل
                    new_delay = current_delay * (1 + adaptation_rate)
                    self.adaptive_parameters['click_delay'] = min(2.0, new_delay)
            
        except Exception as e:
            logging.warning(f"خطأ في تكييف المعاملات: {e}")
    
    def learn_from_click_pattern(self, click_position: Tuple[int, int], element_type: str, 
                               success: bool, confidence: float, context: Dict = None):
        """التعلم من أنماط النقر"""
        try:
            current_time = time.time()
            
            click_record = {
                'position': click_position,
                'element_type': element_type,
                'success': success,
                'confidence': confidence,
                'timestamp': current_time,
                'context': context or {}
            }
            
            self.success_patterns['click_positions'].append(click_record)
            
            # تحديث عتبة الثقة بناءً على النجاح
            if success and confidence > self.adaptive_parameters['confidence_threshold']:
                # زيادة عتبة الثقة تدريجياً للحصول على دقة أفضل
                current_threshold = self.adaptive_parameters['confidence_threshold']
                new_threshold = current_threshold + (confidence - current_threshold) * self.config.adaptation_speed
                self.adaptive_parameters['confidence_threshold'] = min(0.95, new_threshold)
            
            elif not success:
                # تقليل عتبة الثقة عند الفشل
                current_threshold = self.adaptive_parameters['confidence_threshold']
                new_threshold = current_threshold * (1 - self.config.adaptation_speed)
                self.adaptive_parameters['confidence_threshold'] = max(0.5, new_threshold)
            
            logging.debug(f"تعلم من نقر {element_type}: موقع {click_position}، نجاح: {success}")
            
        except Exception as e:
            logging.warning(f"خطأ في التعلم من نمط النقر: {e}")
    
    def get_optimal_timeout(self, event_type: str) -> float:
        """الحصول على المهلة المثلى لنوع حدث معين"""
        try:
            if event_type == 'verified':
                return self.adaptive_parameters['verified_timeout']
            elif event_type == 'success':
                return self.adaptive_parameters['success_timeout']
            else:
                return self.adaptive_parameters.get(f'{event_type}_timeout', 10.0)
        except:
            return 10.0
    
    def get_optimal_click_delay(self, element_type: str = None) -> float:
        """الحصول على تأخير النقر الأمثل"""
        try:
            base_delay = self.adaptive_parameters['click_delay']
            
            # تعديل بناءً على نوع العنصر
            if element_type == 'recaptcha':
                return base_delay * 1.2  # تأخير أطول للكابتشا
            elif element_type in ['verified', 'done', 'google']:
                return base_delay * 0.8  # تأخير أقل للعناصر السريعة
            
            return base_delay
        except:
            return 0.3
    
    def get_optimal_confidence_threshold(self, element_type: str = None) -> float:
        """الحصول على عتبة الثقة المثلى"""
        try:
            base_threshold = self.adaptive_parameters['confidence_threshold']
            
            # تعديل بناءً على نوع العنصر
            if element_type == 'recaptcha':
                return max(0.6, base_threshold - 0.1)  # عتبة أقل للكابتشا
            elif element_type in ['verified', 'done', 'google']:
                return min(0.9, base_threshold + 0.1)  # عتبة أعلى للعناصر الواضحة
            
            return base_threshold
        except:
            return 0.7
    
    def predict_optimal_click_position(self, element_type: str, detected_position: Tuple[int, int], 
                                     confidence: float) -> Tuple[int, int]:
        """التنبؤ بموقع النقر الأمثل بناءً على التعلم"""
        try:
            # البحث عن أنماط ناجحة مشابهة
            successful_clicks = [
                record for record in self.success_patterns['click_positions']
                if record['element_type'] == element_type and record['success'] and 
                record['confidence'] >= confidence * 0.9
            ]
            
            if len(successful_clicks) >= self.config.min_samples_for_learning:
                # حساب متوسط الإزاحة من المواقع الناجحة
                base_x, base_y = detected_position
                
                offsets_x = []
                offsets_y = []
                weights = []
                
                for record in successful_clicks[-20:]:  # آخر 20 نقرة ناجحة
                    record_x, record_y = record['position']
                    
                    # حساب الوزن بناءً على الوقت والثقة
                    time_weight = self.config.time_decay_factor ** (
                        (time.time() - record['timestamp']) / 86400  # تقليل الوزن مع الوقت
                    )
                    confidence_weight = record['confidence']
                    total_weight = time_weight * confidence_weight
                    
                    # حساب الإزاحة النسبية (افتراض أن الكشف الحالي مرجع)
                    offset_x = record_x - base_x
                    offset_y = record_y - base_y
                    
                    offsets_x.append(offset_x)
                    offsets_y.append(offset_y)
                    weights.append(total_weight)
                
                if weights:
                    # حساب متوسط مرجح للإزاحة
                    avg_offset_x = np.average(offsets_x, weights=weights)
                    avg_offset_y = np.average(offsets_y, weights=weights)
                    
                    # تطبيق الإزاحة المتعلمة
                    optimized_x = int(base_x + avg_offset_x)
                    optimized_y = int(base_y + avg_offset_y)
                    
                    # التأكد من أن الموقع معقول (ضمن نطاق محدود)
                    max_offset = 50
                    optimized_x = max(base_x - max_offset, min(base_x + max_offset, optimized_x))
                    optimized_y = max(base_y - max_offset, min(base_y + max_offset, optimized_y))
                    
                    logging.debug(f"موقع محسن لـ {element_type}: ({optimized_x}, {optimized_y}) "
                                f"إزاحة: ({avg_offset_x:.1f}, {avg_offset_y:.1f})")
                    
                    return (optimized_x, optimized_y)
            
            # إذا لم توجد بيانات كافية، استخدم الموقع المكتشف مع تشويش صغير
            noise_x = np.random.randint(-3, 4)
            noise_y = np.random.randint(-3, 4)
            return (detected_position[0] + noise_x, detected_position[1] + noise_y)
            
        except Exception as e:
            logging.warning(f"خطأ في التنبؤ بموقع النقر: {e}")
            return detected_position
    
    def get_learning_statistics(self) -> Dict:
        """الحصول على إحصائيات التعلم"""
        try:
            stats = {
                'total_samples': self.learning_stats['total_samples'],
                'successful_adaptations': self.learning_stats['successful_adaptations'],
                'failed_adaptations': self.learning_stats['failed_adaptations'],
                'adaptation_rate': 0.0,
                'current_parameters': self.adaptive_parameters.copy(),
                'pattern_counts': {
                    key: len(values) for key, values in self.timing_patterns.items()
                },
                'success_pattern_counts': {
                    key: len(values) for key, values in self.success_patterns.items()
                }
            }
            
            total_adaptations = stats['successful_adaptations'] + stats['failed_adaptations']
            if total_adaptations > 0:
                stats['adaptation_rate'] = stats['successful_adaptations'] / total_adaptations
            
            # إحصائيات الأوقات
            if self.timing_patterns['verified_wait']:
                recent_verified = list(self.timing_patterns['verified_wait'])[-10:]
                successful_verified = [r for r in recent_verified if r['success']]
                if successful_verified:
                    stats['avg_verified_time'] = np.mean([r['duration'] for r in successful_verified])
            
            if self.timing_patterns['success_wait']:
                recent_success = list(self.timing_patterns['success_wait'])[-10:]
                successful_success = [r for r in recent_success if r['success']]
                if successful_success:
                    stats['avg_success_time'] = np.mean([r['duration'] for r in successful_success])
            
            return stats
            
        except Exception as e:
            logging.error(f"خطأ في حساب إحصائيات التعلم: {e}")
            return {'total_samples': 0, 'adaptation_rate': 0.0}
    
    def should_save_learning_data(self) -> bool:
        """تحديد ما إذا كان يجب حفظ بيانات التعلم"""
        try:
            auto_save_interval = int(self.config_manager.config.get('LEARNING', 'auto_save_interval', fallback='300'))
            
            if self.learning_stats['last_adaptation'] is None:
                return False
            
            time_since_last_save = time.time() - self.learning_stats.get('last_save', 0)
            return time_since_last_save > auto_save_interval
            
        except:
            return False
    
    def auto_save_if_needed(self):
        """حفظ تلقائي إذا لزم الأمر"""
        try:
            if self.should_save_learning_data():
                self._save_learning_data()
                self.learning_stats['last_save'] = time.time()
                logging.info("تم الحفظ التلقائي لبيانات التعلم")
        except Exception as e:
            logging.warning(f"خطأ في الحفظ التلقائي: {e}")

# ================================
# Advanced Feature Extractor
# ================================

class AdvancedFeatureExtractor:
    """مستخرج الميزات المتقدم"""
    
    def __init__(self):
        self.lbp_radius = 3
        self.lbp_n_points = 24
        self.hog_orientations = 9
        self.hog_pixels_per_cell = (8, 8)
        self.hog_cells_per_block = (2, 2)
    
    def extract_comprehensive_features(self, image: np.ndarray) -> Dict[str, float]:
        """استخراج ميزات شاملة ومتقدمة"""
        try:
            if len(image.shape) == 3:
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            else:
                gray = image.copy()
            
            features = {}
            
            # Basic statistical features
            features.update(self._extract_statistical_features(gray))
            
            # Texture features (LBP)
            features.update(self._extract_lbp_features(gray))
            
            # Shape features (HOG)
            features.update(self._extract_hog_features(gray))
            
            # Edge and contour features
            features.update(self._extract_edge_features(gray))
            
            # Color features (if color image)
            if len(image.shape) == 3:
                features.update(self._extract_color_features(image))
            
            # Frequency domain features
            features.update(self._extract_frequency_features(gray))
            
            return features
            
        except Exception as e:
            logging.error(f"خطأ في استخراج الميزات: {e}")
            return self._get_default_features()
    
    def _extract_statistical_features(self, gray: np.ndarray) -> Dict[str, float]:
        """استخراج الميزات الإحصائية"""
        return {
            'mean_intensity': float(np.mean(gray)),
            'std_intensity': float(np.std(gray)),
            'min_intensity': float(np.min(gray)),
            'max_intensity': float(np.max(gray)),
            'median_intensity': float(np.median(gray)),
            'skewness': float(self._calculate_skewness(gray)),
            'kurtosis': float(self._calculate_kurtosis(gray)),
            'entropy': float(self._calculate_entropy(gray))
        }
    
    def _extract_lbp_features(self, gray: np.ndarray) -> Dict[str, float]:
        """استخراج ميزات Local Binary Pattern"""
        try:
            lbp = local_binary_pattern(gray, self.lbp_n_points, self.lbp_radius, method='uniform')
            hist, _ = np.histogram(lbp.ravel(), bins=self.lbp_n_points + 2, 
                                 range=(0, self.lbp_n_points + 2))
            hist = hist.astype(float)
            hist /= (hist.sum() + 1e-7)
            
            features = {}
            for i, val in enumerate(hist):
                features[f'lbp_bin_{i}'] = val
            
            return features
        except Exception as e:
            logging.warning(f"خطأ في استخراج LBP: {e}")
            return {f'lbp_bin_{i}': 0.0 for i in range(self.lbp_n_points + 2)}
    
    def _extract_hog_features(self, gray: np.ndarray) -> Dict[str, float]:
        """استخراج ميزات Histogram of Oriented Gradients"""
        try:
            resized = cv2.resize(gray, (64, 64))
            
            hog_features = hog(resized, 
                             orientations=self.hog_orientations,
                             pixels_per_cell=self.hog_pixels_per_cell,
                             cells_per_block=self.hog_cells_per_block,
                             visualize=False)
            
            important_features = hog_features[:20]
            
            features = {}
            for i, val in enumerate(important_features):
                features[f'hog_feature_{i}'] = float(val)
            
            return features
        except Exception as e:
            logging.warning(f"خطأ في استخراج HOG: {e}")
            return {f'hog_feature_{i}': 0.0 for i in range(20)}
    
    def _extract_edge_features(self, gray: np.ndarray) -> Dict[str, float]:
        """استخراج ميزات الحواف"""
        try:
            edges = cv2.Canny(gray, 50, 150)
            edge_density = np.sum(edges > 0) / edges.size
            
            sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
            sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
            gradient_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
            
            contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            return {
                'edge_density': float(edge_density),
                'gradient_mean': float(np.mean(gradient_magnitude)),
                'gradient_std': float(np.std(gradient_magnitude)),
                'contour_count': float(len(contours)),
                'largest_contour_area': float(cv2.contourArea(max(contours, key=cv2.contourArea)) if contours else 0),
                'total_contour_area': float(sum(cv2.contourArea(c) for c in contours))
            }
        except Exception as e:
            logging.warning(f"خطأ في استخراج ميزات الحواف: {e}")
            return {
                'edge_density': 0.0,
                'gradient_mean': 0.0,
                'gradient_std': 0.0,
                'contour_count': 0.0,
                'largest_contour_area': 0.0,
                'total_contour_area': 0.0
            }
    
    def _extract_color_features(self, image: np.ndarray) -> Dict[str, float]:
        """استخراج ميزات الألوان"""
        try:
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            
            features = {}
            
            # BGR channels
            for i, channel in enumerate(['b', 'g', 'r']):
                features[f'{channel}_mean'] = float(np.mean(image[:, :, i]))
                features[f'{channel}_std'] = float(np.std(image[:, :, i]))
            
            # HSV channels
            for i, channel in enumerate(['h', 's', 'v']):
                features[f'{channel}_mean'] = float(np.mean(hsv[:, :, i]))
                features[f'{channel}_std'] = float(np.std(hsv[:, :, i]))
            
            # Color histogram
            hist_b = cv2.calcHist([image], [0], None, [8], [0, 256])
            hist_g = cv2.calcHist([image], [1], None, [8], [0, 256])
            hist_r = cv2.calcHist([image], [2], None, [8], [0, 256])
            
            for i in range(8):
                features[f'color_hist_b_{i}'] = float(hist_b[i][0])
                features[f'color_hist_g_{i}'] = float(hist_g[i][0])
                features[f'color_hist_r_{i}'] = float(hist_r[i][0])
            
            return features
        except Exception as e:
            logging.warning(f"خطأ في استخراج ميزات الألوان: {e}")
            return {f'color_feature_{i}': 0.0 for i in range(30)}
    
    def _extract_frequency_features(self, gray: np.ndarray) -> Dict[str, float]:
        """استخراج ميزات المجال الترددي"""
        try:
            f_transform = np.fft.fft2(gray)
            f_shift = np.fft.fftshift(f_transform)
            magnitude_spectrum = np.abs(f_shift)
            
            center = np.array(magnitude_spectrum.shape) // 2
            radius = min(center) // 4
            
            y, x = np.ogrid[:magnitude_spectrum.shape[0], :magnitude_spectrum.shape[1]]
            mask = (x - center[1])**2 + (y - center[0])**2 <= radius**2
            
            low_freq_energy = np.sum(magnitude_spectrum[mask])
            high_freq_energy = np.sum(magnitude_spectrum[~mask])
            total_energy = low_freq_energy + high_freq_energy
            
            return {
                'low_freq_ratio': float(low_freq_energy / (total_energy + 1e-7)),
                'high_freq_ratio': float(high_freq_energy / (total_energy + 1e-7)),
                'freq_energy_total': float(total_energy),
                'freq_energy_mean': float(np.mean(magnitude_spectrum)),
                'freq_energy_std': float(np.std(magnitude_spectrum))
            }
        except Exception as e:
            logging.warning(f"خطأ في استخراج ميزات الترددات: {e}")
            return {
                'low_freq_ratio': 0.0,
                'high_freq_ratio': 0.0,
                'freq_energy_total': 0.0,
                'freq_energy_mean': 0.0,
                'freq_energy_std': 0.0
            }
    
    def _calculate_skewness(self, data: np.ndarray) -> float:
        """حساب الانحراف"""
        try:
            mean = np.mean(data)
            std = np.std(data)
            if std == 0:
                return 0.0
            return np.mean(((data - mean) / std) ** 3)
        except:
            return 0.0
    
    def _calculate_kurtosis(self, data: np.ndarray) -> float:
        """حساب التفلطح"""
        try:
            mean = np.mean(data)
            std = np.std(data)
            if std == 0:
                return 0.0
            return np.mean(((data - mean) / std) ** 4) - 3
        except:
            return 0.0
    
    def _calculate_entropy(self, data: np.ndarray) -> float:
        """حساب الإنتروبيا"""
        try:
            hist, _ = np.histogram(data, bins=256, range=(0, 256))
            hist = hist[hist > 0]
            prob = hist / np.sum(hist)
            return -np.sum(prob * np.log2(prob + 1e-7))
        except:
            return 0.0
    
    def _get_default_features(self) -> Dict[str, float]:
        """ميزات افتراضية في حالة الخطأ"""
        return {f'feature_{i}': 0.0 for i in range(50)}

# ================================
# Advanced ML Engine
# ================================

class AdvancedMLEngine:
    """محرك التعلم الآلي المتقدم"""
    
    def __init__(self, config_manager: SecureConfigManager):
        self.config_manager = config_manager
        self.feature_extractor = AdvancedFeatureExtractor()
        
        # نماذج متعددة للتنبؤ
        self.models = {
            'region_classifier': RandomForestClassifier(
                n_estimators=100,
                max_depth=10,
                random_state=42,
                n_jobs=-1
            ),
            'success_predictor': MLPClassifier(
                hidden_layer_sizes=(64, 32, 16),
                max_iter=1000,
                random_state=42,
                early_stopping=True
            ),
            'click_optimizer': RandomForestClassifier(
                n_estimators=50,
                max_depth=8,
                random_state=42
            ),
            'anomaly_detector': IsolationForest(
                contamination=0.1,
                random_state=42
            )
        }
        
        # معالجات البيانات
        self.scalers = {
            'standard': StandardScaler(),
            'robust': RobustScaler()
        }
        
        self.feature_selector = SelectKBest(
            score_func=f_classif,
            k=int(self.config_manager.config.get('ML', 'feature_selection_k', fallback='15'))
        )
        
        # إحصائيات النماذج
        self.model_stats = {
            'training_samples': 0,
            'accuracy_scores': {},
            'confidence_intervals': {},
            'last_training': None,
            'feature_importance': {},
            'cross_val_scores': {}
        }
        
        self.is_trained = False
        self._load_models()
    
    def _load_models(self):
        """تحميل النماذج المحفوظة"""
        try:
            models_path = self.config_manager.get_secure_path('PATHS', 'memory_dir', 'advanced_models.pkl')
            
            if models_path.exists():
                with open(models_path, 'rb') as f:
                    saved_data = pickle.load(f)
                    
                self.models = saved_data.get('models', self.models)
                self.scalers = saved_data.get('scalers', self.scalers)
                self.feature_selector = saved_data.get('feature_selector', self.feature_selector)
                self.model_stats = saved_data.get('model_stats', self.model_stats)
                self.is_trained = saved_data.get('is_trained', False)
                
                logging.info("تم تحميل النماذج المتقدمة بنجاح")
            else:
                logging.info("لا توجد نماذج محفوظة، سيتم البدء بنماذج جديدة")
                
        except Exception as e:
            logging.error(f"خطأ في تحميل النماذج: {e}")
            self.is_trained = False
    
    def _save_models(self):
        """حفظ النماذج"""
        try:
            models_path = self.config_manager.get_secure_path('PATHS', 'memory_dir', 'advanced_models.pkl')
            
            save_data = {
                'models': self.models,
                'scalers': self.scalers,
                'feature_selector': self.feature_selector,
                'model_stats': self.model_stats,
                'is_trained': self.is_trained,
                'save_timestamp': datetime.now().isoformat()
            }
            
            with open(models_path, 'wb') as f:
                pickle.dump(save_data, f)
                
            logging.info("تم حفظ النماذج المتقدمة")
            
        except Exception as e:
            logging.error(f"خطأ في حفظ النماذج: {e}")
    
    def prepare_training_data(self, training_samples: List[Dict]) -> Tuple[np.ndarray, np.ndarray, List[str]]:
        """إعداد بيانات التدريب المتقدمة"""
        try:
            min_samples = int(self.config_manager.config.get('ML', 'min_training_samples', fallback='50'))
            if len(training_samples) < min_samples:
                raise ValueError(f"عدد العينات غير كافي: {len(training_samples)}")
            
            features_list = []
            labels = []
            feature_names = []
            
            for sample in training_samples:
                if 'advanced_features' in sample and sample['advanced_features']:
                    features_dict = sample['advanced_features']
                    
                    if not feature_names:
                        feature_names = sorted(features_dict.keys())
                    
                    feature_vector = [features_dict.get(name, 0.0) for name in feature_names]
                    features_list.append(feature_vector)
                    
                    # تحويل النتيجة إلى تصنيف
                    result = sample.get('result', 'unknown')
                    if result == 'success':
                        labels.append(1)
                    elif result == 'fail':
                        labels.append(0)
                    else:
                        labels.append(-1)  # غير معروف
            
            if not features_list:
                raise ValueError("لا توجد ميزات صالحة للتدريب")
            
            X = np.array(features_list)
            y = np.array(labels)
            
            # إزالة العينات غير المعروفة
            valid_indices = y != -1
            X = X[valid_indices]
            y = y[valid_indices]
            
            if len(X) == 0:
                raise ValueError("لا توجد عينات صالحة بعد التنظيف")
            
            # معالجة القيم المفقودة
            X = np.nan_to_num(X, nan=0.0, posinf=1.0, neginf=-1.0)
            
            return X, y, feature_names
            
        except Exception as e:
            logging.error(f"خطأ في إعداد بيانات التدريب: {e}")
            raise
    
    def train_models(self, training_samples: List[Dict]) -> Dict[str, float]:
        """تدريب النماذج المتقدمة مع إحصائيات متقدمة"""
        try:
            logging.info(f"بدء تدريب النماذج على {len(training_samples)} عينة")
            
            X, y, feature_names = self.prepare_training_data(training_samples)
            
            # تطبيع البيانات
            X_scaled = self.scalers['robust'].fit_transform(X)
            
            # اختيار الميزات المهمة
            k_features = int(self.config_manager.config.get('ML', 'feature_selection_k', fallback='15'))
            if len(feature_names) > k_features:
                X_selected = self.feature_selector.fit_transform(X_scaled, y)
                selected_features = self.feature_selector.get_support()
                logging.info(f"تم اختيار {np.sum(selected_features)} ميزة من {len(feature_names)}")
            else:
                X_selected = X_scaled
            
            # تدريب النماذج
            results = {}
            confidence_intervals = {}
            
            # تدريب مصنف المناطق
            if len(np.unique(y)) > 1:
                self.models['region_classifier'].fit(X_selected, y)
                
                # تقييم متقاطع مع فترات الثقة
                cv_folds = int(self.config_manager.config.get('ML', 'cross_validation_folds', fallback='5'))
                cv_scores = cross_val_score(self.models['region_classifier'], X_selected, y, cv=cv_folds)
                
                mean_score, ci_low, ci_high = calculate_confidence_interval(cv_scores)
                results['region_classifier'] = mean_score
                confidence_intervals['region_classifier'] = (ci_low, ci_high)
                self.model_stats['cross_val_scores']['region_classifier'] = cv_scores.tolist()
                
                # تدريب متنبئ النجاح
                self.models['success_predictor'].fit(X_selected, y)
                cv_scores_mlp = cross_val_score(self.models['success_predictor'], X_selected, y, cv=cv_folds)
                
                mean_score_mlp, ci_low_mlp, ci_high_mlp = calculate_confidence_interval(cv_scores_mlp)
                results['success_predictor'] = mean_score_mlp
                confidence_intervals['success_predictor'] = (ci_low_mlp, ci_high_mlp)
                self.model_stats['cross_val_scores']['success_predictor'] = cv_scores_mlp.tolist()
                
                # تدريب محسن النقر
                self.models['click_optimizer'].fit(X_selected, y)
                cv_scores_opt = cross_val_score(self.models['click_optimizer'], X_selected, y, cv=cv_folds)
                
                mean_score_opt, ci_low_opt, ci_high_opt = calculate_confidence_interval(cv_scores_opt)
                results['click_optimizer'] = mean_score_opt
                confidence_intervals['click_optimizer'] = (ci_low_opt, ci_high_opt)
                self.model_stats['cross_val_scores']['click_optimizer'] = cv_scores_opt.tolist()
            
            # تدريب كاشف الشذوذ
            self.models['anomaly_detector'].fit(X_selected)
            
            # حفظ إحصائيات التدريب
            self.model_stats['training_samples'] = len(training_samples)
            self.model_stats['last_training'] = datetime.now().isoformat()
            self.model_stats['accuracy_scores'] = results
            self.model_stats['confidence_intervals'] = confidence_intervals
            
            # حفظ أهمية الميزات
            if hasattr(self.models['region_classifier'], 'feature_importances_'):
                importances = self.models['region_classifier'].feature_importances_
                if len(feature_names) == len(importances):
                    self.model_stats['feature_importance'] = dict(zip(feature_names, importances.tolist()))
            
            self.is_trained = True
            self._save_models()
            
            logging.info(f"تم تدريب النماذج بنجاح. النتائج: {results}")
            if HAS_STATSMODELS:
                logging.info(f"فترات الثقة: {confidence_intervals}")
            
            return results
            
        except Exception as e:
            logging.error(f"خطأ في تدريب النماذج: {e}")
            return {}
    
    def predict_best_regions(self, image: np.ndarray, num_regions: int = 5) -> List[Dict]:
        """التنبؤ بأفضل المناطق للنقر"""
        try:
            if not self.is_trained:
                logging.warning("النماذج غير مدربة، سيتم استخدام التنبؤ الافتراضي")
                return self._get_default_predictions(image, num_regions)
            
            # استخراج الميزات
            features = self.feature_extractor.extract_comprehensive_features(image)
            feature_vector = np.array([list(features.values())]).reshape(1, -1)
            
            # تطبيع البيانات
            feature_vector_scaled = self.scalers['robust'].transform(feature_vector)
            
            # اختيار الميزات
            if hasattr(self.feature_selector, 'transform'):
                feature_vector_selected = self.feature_selector.transform(feature_vector_scaled)
            else:
                feature_vector_selected = feature_vector_scaled
            
            predictions = []
            
            # التنبؤ باستخدام النماذج المختلفة
            for model_name, model in self.models.items():
                if model_name == 'anomaly_detector':
                    continue
                
                try:
                    if hasattr(model, 'predict_proba'):
                        proba = model.predict_proba(feature_vector_selected)[0]
                        confidence = np.max(proba)
                        prediction = np.argmax(proba)
                    else:
                        prediction = model.predict(feature_vector_selected)[0]
                        confidence = 0.5
                    
                    predictions.append({
                        'model': model_name,
                        'prediction': int(prediction),
                        'confidence': float(confidence),
                        'features_used': len(feature_vector_selected[0])
                    })
                    
                except Exception as e:
                    logging.warning(f"خطأ في التنبؤ بالنموذج {model_name}: {e}")
            
            # دمج التنبؤات
            ensemble_predictions = self._ensemble_predictions(predictions, image)
            
            return ensemble_predictions[:num_regions]
            
        except Exception as e:
            logging.error(f"خطأ في التنبؤ بالمناطق: {e}")
            return self._get_default_predictions(image, num_regions)
    
    def _ensemble_predictions(self, predictions: List[Dict], image: np.ndarray) -> List[Dict]:
        """دمج تنبؤات النماذج المختلفة"""
        try:
            h, w = image.shape[:2]
            
            # إنشاء مناطق مرشحة بناءً على التنبؤات
            candidate_regions = []
            
            # مناطق مركزية (عادة مفيدة للكابتشا)
            center_regions = [
                {'x': w//2, 'y': h//2, 'source': 'center', 'base_confidence': 0.7},
                {'x': w//3, 'y': h//2, 'source': 'left_center', 'base_confidence': 0.6},
                {'x': 2*w//3, 'y': h//2, 'source': 'right_center', 'base_confidence': 0.6},
                {'x': w//2, 'y': h//3, 'source': 'top_center', 'base_confidence': 0.5},
                {'x': w//2, 'y': 2*h//3, 'source': 'bottom_center', 'base_confidence': 0.5}
            ]
            
            # تعديل الثقة بناءً على تنبؤات النماذج
            for region in center_regions:
                total_confidence = region['base_confidence']
                model_count = 0
                
                for pred in predictions:
                    if pred['prediction'] == 1:  # تنبؤ إيجابي
                        total_confidence += pred['confidence'] * 0.3
                        model_count += 1
                
                # تطبيع الثقة
                if model_count > 0:
                    total_confidence = total_confidence / (1 + model_count * 0.3)
                
                region['confidence'] = min(0.95, total_confidence)
                region['models_agreement'] = model_count
                
                candidate_regions.append(region)
            
            # ترتيب حسب الثقة
            candidate_regions.sort(key=lambda x: x['confidence'], reverse=True)
            
            return candidate_regions
            
        except Exception as e:
            logging.error(f"خطأ في دمج التنبؤات: {e}")
            return self._get_default_predictions(image, 5)
    
    def _get_default_predictions(self, image: np.ndarray, num_regions: int) -> List[Dict]:
        """تنبؤات افتراضية في حالة عدم وجود نماذج مدربة"""
        h, w = image.shape[:2]
        
        default_regions = [
            {'x': w//2, 'y': h//2, 'confidence': 0.5, 'source': 'default_center'},
            {'x': w//3, 'y': h//2, 'confidence': 0.4, 'source': 'default_left'},
            {'x': 2*w//3, 'y': h//2, 'confidence': 0.4, 'source': 'default_right'},
            {'x': w//2, 'y': h//3, 'confidence': 0.3, 'source': 'default_top'},
            {'x': w//2, 'y': 2*h//3, 'confidence': 0.3, 'source': 'default_bottom'}
        ]
        
        return default_regions[:num_regions]
    
    def detect_anomaly(self, features: Dict[str, float]) -> Dict[str, Union[bool, float]]:
        """كشف الشذوذ في الميزات"""
        try:
            if not self.is_trained or 'anomaly_detector' not in self.models:
                return {'is_anomaly': False, 'anomaly_score': 0.0}
            
            feature_vector = np.array([list(features.values())]).reshape(1, -1)
            feature_vector_scaled = self.scalers['robust'].transform(feature_vector)
            
            # كشف الشذوذ
            anomaly_prediction = self.models['anomaly_detector'].predict(feature_vector_scaled)[0]
            anomaly_score = self.models['anomaly_detector'].score_samples(feature_)
            anomaly_score = self.models['anomaly_detector'].score_samples(feature_vector_scaled)[0]
            
            return {
                'is_anomaly': anomaly_prediction == -1,
                'anomaly_score': float(anomaly_score),
                'confidence': abs(float(anomaly_score))
            }
            
        except Exception as e:
            logging.warning(f"خطأ في كشف الشذوذ: {e}")
            return {'is_anomaly': False, 'anomaly_score': 0.0}

# ================================
# Advanced Template Matcher
# ================================

class AdvancedTemplateMatcher:
    """مطابق القوالب المتقدم مع مقاومة التغييرات"""
    
    def __init__(self, config_manager: SecureConfigManager):
        self.config_manager = config_manager
        self.templates = {}
        self.template_features = {}
        self.matching_cache = {}
        self.cache_timeout = 1.0
        
        # إعدادات المطابقة المتقدمة
        self.scale_range = np.linspace(0.7, 1.3, 7)
        self.rotation_range = [-5, -2, 0, 2, 5]
        self.methods = [
            cv2.TM_CCOEFF_NORMED,
            cv2.TM_CCORR_NORMED,
            cv2.TM_SQDIFF_NORMED
        ]
        
        self.feature_extractor = AdvancedFeatureExtractor()
        self._load_templates()
    
    def _load_templates(self):
        """تحميل القوالب مع معالجة الأخطاء المتقدمة"""
        template_names = ['recaptcha', 'verified', 'done', 'google']
        
        for name in template_names:
            try:
                template_path = self.config_manager.get_template_path(name)
                
                if template_path.exists():
                    template = cv2.imread(str(template_path), cv2.IMREAD_COLOR)
                    if template is not None:
                        self.templates[name] = template
                        self.template_features[name] = self.feature_extractor.extract_comprehensive_features(template)
                        logging.info(f"تم تحميل قالب {name} بنجاح")
                    else:
                        logging.warning(f"فشل في قراءة قالب {name}")
                        self._create_fallback_template(name)
                else:
                    logging.warning(f"قالب {name} غير موجود في {template_path}")
                    self._create_fallback_template(name)
                    
            except Exception as e:
                logging.error(f"خطأ في تحميل قالب {name}: {e}")
                self._create_fallback_template(name)
    
    def _create_fallback_template(self, name: str):
        """إنشاء قالب احتياطي"""
        try:
            fallback_template = np.zeros((50, 100, 3), dtype=np.uint8)
            cv2.putText(fallback_template, name.upper(), (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            self.templates[name] = fallback_template
            self.template_features[name] = self.feature_extractor.extract_comprehensive_features(fallback_template)
            
            logging.info(f"تم إنشاء قالب احتياطي لـ {name}")
            
        except Exception as e:
            logging.error(f"فشل في إنشاء قالب احتياطي لـ {name}: {e}")
    
    def find_template_advanced(self, template_name: str, image: np.ndarray, 
                             threshold: float = 0.8) -> Optional[Dict]:
        """البحث المتقدم عن القالب مع مقاومة التغييرات"""
        try:
            if template_name not in self.templates:
                logging.warning(f"قالب {template_name} غير موجود")
                return None
            
            # فحص الكاش
            cache_key = f"{template_name}_{hash(image.tobytes())}"
            current_time = time.time()
            
            if cache_key in self.matching_cache:
                cached_result = self.matching_cache[cache_key]
                if current_time - cached_result['timestamp'] < self.cache_timeout:
                    return cached_result['result']
            
            template = self.templates[template_name]
            best_match = None
            best_confidence = 0
            
            # البحث مع تحويلات متعددة
            for scale in self.scale_range:
                for rotation in self.rotation_range:
                    try:
                        transformed_template = self._transform_template(template, scale, rotation)
                        
                        if transformed_template is None:
                            continue
                        
                        for method in self.methods:
                            match_result = self._match_template_method(
                                image, transformed_template, method, threshold
                            )
                            
                            if match_result and match_result['confidence'] > best_confidence:
                                best_match = match_result
                                best_match['scale'] = scale
                                best_match['rotation'] = rotation
                                best_match['method'] = method
                                best_confidence = match_result['confidence']
                                
                    except Exception as e:
                        continue
            
            # التحقق من المطابقة باستخدام الميزات
            if best_match:
                feature_similarity = self._verify_match_with_features(
                    image, best_match, template_name
                )
                best_match['feature_similarity'] = feature_similarity
                best_match['final_confidence'] = (
                    best_match['confidence'] * 0.7 + feature_similarity * 0.3
                )
            
            # حفظ في الكاش
            self.matching_cache[cache_key] = {
                'result': best_match,
                'timestamp': current_time
            }
            
            self._cleanup_cache()
            
            return best_match
            
        except Exception as e:
            logging.error(f"خطأ في البحث المتقدم عن {template_name}: {e}")
            return None
    
    def _transform_template(self, template: np.ndarray, scale: float, rotation: float) -> Optional[np.ndarray]:
        """تطبيق التحويلات على القالب"""
        try:
            h, w = template.shape[:2]
            
            new_w, new_h = int(w * scale), int(h * scale)
            if new_w <= 0 or new_h <= 0:
                return None
            
            scaled = cv2.resize(template, (new_w, new_h))
            
            if abs(rotation) > 0.1:
                center = (new_w // 2, new_h // 2)
                rotation_matrix = cv2.getRotationMatrix2D(center, rotation, 1.0)
                rotated = cv2.warpAffine(scaled, rotation_matrix, (new_w, new_h))
                return rotated
            
            return scaled
            
        except Exception as e:
            return None
    
    def _match_template_method(self, image: np.ndarray, template: np.ndarray, 
                             method: int, threshold: float) -> Optional[Dict]:
        """مطابقة القالب بطريقة محددة"""
        try:
            if template.shape[0] > image.shape[0] or template.shape[1] > image.shape[1]:
                return None
            
            result = cv2.matchTemplate(image, template, method)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
            
            if method == cv2.TM_SQDIFF_NORMED:
                confidence = 1 - min_val
                location = min_loc
            else:
                confidence = max_val
                location = max_loc
            
            if confidence >= threshold:
                h, w = template.shape[:2]
                center_x = location[0] + w // 2
                center_y = location[1] + h // 2
                
                return {
                    'x': center_x,
                    'y': center_y,
                    'width': w,
                    'height': h,
                    'confidence': float(confidence),
                    'location': location,
                    'template_size': (w, h)
                }
            
            return None
            
        except Exception as e:
            return None
    
    def _verify_match_with_features(self, image: np.ndarray, match: Dict, template_name: str) -> float:
        """التحقق من المطابقة باستخدام الميزات"""
        try:
            x, y = match['location']
            w, h = match['template_size']
            
            if x + w > image.shape[1] or y + h > image.shape[0]:
                return 0.0
            
            matched_region = image[y:y+h, x:x+w]
            region_features = self.feature_extractor.extract_comprehensive_features(matched_region)
            
            template_features = self.template_features.get(template_name, {})
            
            if not template_features or not region_features:
                return 0.5
            
            similarity = self._calculate_feature_similarity(region_features, template_features)
            return similarity
            
        except Exception as e:
            logging.warning(f"خطأ في التحقق من الميزات: {e}")
            return 0.5
    
    def _calculate_feature_similarity(self, features1: Dict, features2: Dict) -> float:
        """حساب التشابه بين الميزات"""
        try:
            common_keys = set(features1.keys()) & set(features2.keys())
            if not common_keys:
                return 0.0
            
            similarities = []
            for key in common_keys:
                val1, val2 = features1[key], features2[key]
                
                if val1 == 0 and val2 == 0:
                    sim = 1.0
                elif val1 == 0 or val2 == 0:
                    sim = 0.0
                else:
                    diff = abs(val1 - val2)
                    max_val = max(abs(val1), abs(val2))
                    sim = 1.0 - (diff / max_val)
                
                similarities.append(max(0.0, sim))
            
            return np.mean(similarities)
            
        except Exception as e:
            return 0.0
    
    def _cleanup_cache(self):
        """تنظيف كاش المطابقة"""
        try:
            current_time = time.time()
            expired_keys = [
                key for key, value in self.matching_cache.items()
                if current_time - value['timestamp'] > self.cache_timeout * 2
            ]
            
            for key in expired_keys:
                del self.matching_cache[key]
                
        except Exception as e:
            logging.warning(f"خطأ في تنظيف كاش المطابقة: {e}")

# ================================
# Smart Screenshot Manager
# ================================

class SmartScreenshotManager:
    """مدير لقطات الشاشة الذكي"""
    
    def __init__(self, config: PerformanceConfig):
        self.config = config
        self.frame_buffer = deque(maxlen=config.frame_buffer_size)
        self.last_screenshot = None
        self.screenshot_cache = {}
        self.cache_timeout = 0.5
        
    def capture_smart_screenshot(self, force_full: bool = False) -> Optional[np.ndarray]:
        """التقاط لقطة شاشة ذكية"""
        try:
            current_time = time.time()
            
            if not force_full and self.last_screenshot is not None:
                if current_time - self.screenshot_cache.get('timestamp', 0) < self.cache_timeout:
                    return self.last_screenshot
            
            with mss.mss() as sct:
                monitor = sct.monitors[1]
                screenshot = np.array(sct.grab(monitor))
                img = cv2.cvtColor(screenshot, cv2.COLOR_BGRA2BGR)
                
                if self.config.adaptive_quality:
                    img = self._adaptive_resize(img)
                
                self.last_screenshot = img
                self.screenshot_cache = {'timestamp': current_time}
                self.frame_buffer.append({'image': img, 'timestamp': current_time})
                
                return img
                
        except Exception as e:
            logging.error(f"خطأ في التقاط الشاشة: {e}")
            return None
    
    def _adaptive_resize(self, img: np.ndarray) -> np.ndarray:
        """تغيير حجم تكيفي للصورة"""
        try:
            h, w = img.shape[:2]
            max_w, max_h = self.config.max_screenshot_resolution
            
            if w <= max_w and h <= max_h:
                return img
            
            scale_w = max_w / w
            scale_h = max_h / h
            scale = min(scale_w, scale_h)
            
            new_w = int(w * scale)
            new_h = int(h * scale)
            
            resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
            return resized
            
        except Exception as e:
            logging.warning(f"خطأ في تغيير حجم الصورة: {e}")
            return img
    
    def cleanup_cache(self):
        """تنظيف الكاش"""
        try:
            self.screenshot_cache.clear()
            if len(self.frame_buffer) > self.config.frame_buffer_size // 2:
                keep_count = self.config.frame_buffer_size // 2
                self.frame_buffer = deque(list(self.frame_buffer)[-keep_count:], 
                                        maxlen=self.config.frame_buffer_size)
        except Exception as e:
            logging.warning(f"خطأ في تنظيف كاش الصور: {e}")

# ================================
# Intelligent Click Manager
# ================================

class IntelligentClickManager:
    """مدير النقر الذكي مع تعلم الأنماط"""
    
    def __init__(self, config_manager: SecureConfigManager, learning_engine: AdaptiveLearningEngine):
        self.config_manager = config_manager
        self.learning_engine = learning_engine
        self.click_history = deque(maxlen=100)
        self.last_click_time = 0
        
    def smart_click(self, target_info: Dict, element_type: str) -> bool:
        """نقر ذكي مع تعلم الأنماط"""
        try:
            current_time = time.time()
            
            # الحصول على التأخير الأمثل من التعلم
            optimal_delay = self.learning_engine.get_optimal_click_delay(element_type)
            
            if current_time - self.last_click_time < optimal_delay:
                time.sleep(optimal_delay - (current_time - self.last_click_time))
            
            # تحسين نقطة النقر باستخدام التعلم
            base_position = (target_info.get('x', 0), target_info.get('y', 0))
            confidence = target_info.get('confidence', 0.5)
            
            optimized_position = self.learning_engine.predict_optimal_click_position(
                element_type, base_position, confidence
            )
            
            # تنفيذ النقر
            success = self._execute_click(optimized_position, optimal_delay, element_type)
            
            # تسجيل النتيجة للتعلم
            self.learning_engine.learn_from_click_pattern(
                optimized_position, element_type, success, confidence,
                context={'original_position': base_position, 'delay_used': optimal_delay}
            )
            
            # تسجيل في التاريخ
            self._record_click(optimized_position, element_type, success, target_info)
            
            self.last_click_time = time.time()
            
            return success
            
        except Exception as e:
            logging.error(f"خطأ في النقر الذكي: {e}")
            return False
    
    def _execute_click(self, click_point: Tuple[int, int], delay: float, element_type: str) -> bool:
        """تنفيذ النقر مع معالجة الأخطاء"""
        try:
            x, y = click_point
            
            if x < 0 or y < 0 or x > 3840 or y > 2160:
                logging.warning(f"إحداثيات نقر غير صالحة: ({x}, {y})")
                return False
            
            current_x, current_y = pyautogui.position()
            distance = np.sqrt((x - current_x)**2 + (y - current_y)**2)
            move_time = min(delay, distance / 1000)
            
            pyautogui.moveTo(x, y, duration=move_time, tween=pyautogui.easeOutQuad)
            time.sleep(0.05)
            pyautogui.click()
            
            logging.info(f"تم النقر على {element_type} في ({x}, {y}) بتأخير {delay:.3f}s")
            
            return True
            
        except Exception as e:
            logging.error(f"خطأ في تنفيذ النقر: {e}")
            return False
    
    def _record_click(self, click_point: Tuple[int, int], element_type: str, 
                     success: bool, target_info: Dict):
        """تسجيل النقر في التاريخ"""
        try:
            click_record = {
                'timestamp': time.time(),
                'point': click_point,
                'element_type': element_type,
                'success': success,
                'confidence': target_info.get('confidence', 0.0),
                'target_info': target_info.copy()
            }
            
            self.click_history.append(click_record)
            
        except Exception as e:
            logging.warning(f"خطأ في تسجيل النقر: {e}")
    
    def get_click_statistics(self) -> Dict:
        """الحصول على إحصائيات النقر"""
        try:
            stats = {
                'total_clicks': len(self.click_history),
                'success_rate': 0.0,
                'element_stats': {}
            }
            
            if self.click_history:
                successful_clicks = sum(1 for c in self.click_history if c['success'])
                stats['success_rate'] = successful_clicks / len(self.click_history)
            
            for element_type in ['recaptcha', 'verified', 'done', 'google']:
                element_clicks = [c for c in self.click_history if c['element_type'] == element_type]
                
                if element_clicks:
                    element_success = sum(1 for c in element_clicks if c['success'])
                    stats['element_stats'][element_type] = {
                        'total': len(element_clicks),
                        'successful': element_success,
                        'success_rate': element_success / len(element_clicks),
                        'avg_confidence': np.mean([c['confidence'] for c in element_clicks])
                    }
            
            return stats
            
        except Exception as e:
            logging.error(f"خطأ في حساب إحصائيات النقر: {e}")
            return {'total_clicks': 0, 'success_rate': 0.0}

# ================================
# Advanced Wait Manager
# ================================

class AdvancedWaitManager:
    """مدير الانتظار المتقدم مع تعلم الأوقات المثلى"""
    
    def __init__(self, config_manager: SecureConfigManager, template_matcher: AdvancedTemplateMatcher,  learning_engine: AdaptiveLearningEngine):
        self.config_manager = config_manager
        self.template_matcher = template_matcher
        self.learning_engine = learning_engine
        self.wait_history = deque(maxlen=50)
        self.check_interval = 0.2
        
    def smart_wait_for_element(self, target_elements: List[str], timeout: float = None,  context: str = "general") -> Optional[Dict]:
        """انتظار ذكي لظهور عنصر من قائمة العناصر"""
        try:
            if timeout is None:
                timeout = self.learning_engine.get_optimal_timeout(context)
            
            start_time = time.time()
            last_check_time = 0
            check_count = 0
            
            logging.info(f"بدء الانتظار الذكي لـ {target_elements} لمدة {timeout:.1f}s")
            
            while time.time() - start_time < timeout:
                current_time = time.time()
                
                if current_time - last_check_time < self._get_adaptive_check_interval(context, check_count):
                    time.sleep(0.05)
                    continue
                
                try:
                    screenshot = self._capture_for_wait()
                    if screenshot is None:
                        time.sleep(self.check_interval)
                        continue
                    
                    for element_name in target_elements:
                        # استخدام العتبة المثلى من التعلم
                        optimal_threshold = self.learning_engine.get_optimal_confidence_threshold(element_name)
                        
                        detection = self.template_matcher.find_template_advanced(
                            element_name, screenshot, threshold=optimal_threshold
                        )
                        
                        if detection and detection.get('final_confidence', 0) > optimal_threshold:
                            elapsed = time.time() - start_time
                            
                            # تعلم من النجاح
                            self.learning_engine.learn_from_timing(
                                f'{context}_wait', elapsed, True,
                                context={'element': element_name, 'threshold_used': optimal_threshold}
                            )
                            
                            self._record_wait_success(element_name, elapsed, context)
                            
                            logging.info(f"✅ تم العثور على {element_name} خلال {elapsed:.2f}s")
                            
                            return {
                                'element': element_name,
                                'detection': detection,
                                'elapsed_time': elapsed,
                                'success': True
                            }
                    
                    last_check_time = current_time
                    check_count += 1
                    
                    if check_count % 25 == 0:
                        remaining = timeout - (current_time - start_time)
                        logging.info(f"⏳ انتظار {target_elements}... متبقي: {remaining:.1f}s")
                    
                except Exception as e:
                    logging.warning(f"خطأ في فحص الانتظار: {e}")
                    time.sleep(self.check_interval)
                    continue
            
            # انتهاء المهلة
            elapsed = time.time() - start_time
            
            # تعلم من الفشل
            self.learning_engine.learn_from_timing(
                f'{context}_wait', elapsed, False,
                context={'elements': target_elements, 'timeout_used': timeout}
            )
            
            self._record_wait_timeout(target_elements, elapsed, context)
            
            logging.warning(f"⏰ انتهت مهلة انتظار {target_elements} ({elapsed:.1f}s)")
            
            return {
                'element': None,
                'detection': None,
                'elapsed_time': elapsed,
                'success': False,
                'reason': 'timeout'
            }
            
        except Exception as e:
            logging.error(f"خطأ في الانتظار الذكي: {e}")
            return {
                'element': None,
                'detection': None,
                'elapsed_time': 0,
                'success': False,
                'reason': 'error'
            }
    
    def _get_adaptive_check_interval(self, context: str, check_count: int) -> float:
        """حساب فترة الفحص التكيفية"""
        try:
            base_interval = self.check_interval
            
            if check_count < 10:
                return base_interval * 0.5
            
            if check_count > 50:
                return base_interval * 1.5
            
            context_multipliers = {
                'verified': 0.8,
                'success': 1.0,
                'general': 1.2
            }
            
            return base_interval * context_multipliers.get(context, 1.0)
            
        except Exception as e:
            return self.check_interval
    
    def _capture_for_wait(self) -> Optional[np.ndarray]:
        """التقاط محسن للانتظار"""
        try:
            with mss.mss() as sct:
                monitor = sct.monitors[1]
                screenshot = np.array(sct.grab(monitor))
                return cv2.cvtColor(screenshot, cv2.COLOR_BGRA2BGR)
        except Exception as e:
            logging.warning(f"خطأ في التقاط الشاشة للانتظار: {e}")
            return None
    
    def _record_wait_success(self, element: str, elapsed_time: float, context: str):
        """تسجيل نجاح الانتظار"""
        try:
            record = {
                'timestamp': time.time(),
                'element': element,
                'elapsed_time': elapsed_time,
                'context': context,
                'success': True
            }
            
            self.wait_history.append(record)
                
        except Exception as e:
            logging.warning(f"خطأ في تسجيل نجاح الانتظار: {e}")
    
    def _record_wait_timeout(self, elements: List[str], elapsed_time: float, context: str):
        """تسجيل انتهاء مهلة الانتظار"""
        try:
            record = {
                'timestamp': time.time(),
                'elements': elements,
                'elapsed_time': elapsed_time,
                'context': context,
                'success': False,
                'reason': 'timeout'
            }
            
            self.wait_history.append(record)
            
        except Exception as e:
            logging.warning(f"خطأ في تسجيل انتهاء المهلة: {e}")
    
    def get_wait_statistics(self) -> Dict:
        """الحصول على إحصائيات الانتظار"""
        try:
            if not self.wait_history:
                return {'total_waits': 0, 'success_rate': 0.0}
            
            total_waits = len(self.wait_history)
            successful_waits = sum(1 for w in self.wait_history if w.get('success', False))
            
            stats = {
                'total_waits': total_waits,
                'successful_waits': successful_waits,
                'success_rate': successful_waits / total_waits,
                'avg_wait_time': np.mean([w.get('elapsed_time', 0) for w in self.wait_history]),
                'context_stats': {}
            }
            
            for context in ['verified', 'success', 'general']:
                context_waits = [w for w in self.wait_history if w.get('context') == context]
                
                if context_waits:
                    context_success = sum(1 for w in context_waits if w.get('success', False))
                    stats['context_stats'][context] = {
                        'total': len(context_waits),
                        'successful': context_success,
                        'success_rate': context_success / len(context_waits),
                        'avg_time': np.mean([w.get('elapsed_time', 0) for w in context_waits])
                    }
            
            return stats
            
        except Exception as e:
            logging.error(f"خطأ في حساب إحصائيات الانتظار: {e}")
            return {'total_waits': 0, 'success_rate': 0.0}

# ================================
# Main Advanced Captcha Solver
# ================================

class AdvancedCaptchaSolver:
    """حلال الكابتشا المتقدم مع التعلم التكيفي"""
    
    def __init__(self):
        # إعداد نظام السجلات
        self._setup_logging()
        
        # تحميل الإعدادات
        self.config_manager = SecureConfigManager()
        self.security_config = SecurityConfig()
        self.performance_config = PerformanceConfig()
        self.learning_config = LearningConfig()
        
        # مراقب الأداء
        self.performance_monitor = PerformanceMonitor(self.performance_config)
        
        # محرك التعلم التكيفي
        self.learning_engine = AdaptiveLearningEngine(self.learning_config, self.config_manager)
        
        # المكونات الأساسية
        self.ml_engine = AdvancedMLEngine(self.config_manager)
        self.screenshot_manager = SmartScreenshotManager(self.performance_config)
        self.template_matcher = AdvancedTemplateMatcher(self.config_manager)
        self.click_manager = IntelligentClickManager(self.config_manager, self.learning_engine)
        self.wait_manager = AdvancedWaitManager(self.config_manager, self.template_matcher, self.learning_engine)
        
        # حالة النظام
        self.system_state = "initializing"
        self.current_session = {
            'start_time': time.time(),
            'cycles_completed': 0,
            'captchas_solved': 0,
            'errors_encountered': 0,
            'last_activity': time.time(),
            'last_auto_save': time.time()
        }
        
        # إعدادات الحل
        self.solving_strategy = "adaptive"
        self.max_retry_attempts = 3
        self.retry_delay_multiplier = 1.5
                # تحميل البيانات المحفوظة
        self._load_session_data()
        
        logging.info("🚀 تم تهيئة حلال الكابتشا المتقدم مع التعلم التكيفي بنجاح")
    
    def _setup_logging(self):
        """إعداد نظام السجلات المتقدم"""
        try:
            logs_dir = Path("logs")
            logs_dir.mkdir(exist_ok=True)
            
            log_format = '%(asctime)s - %(levelname)s - %(funcName)s:%(lineno)d - %(message)s'
            log_filename = logs_dir / f"captcha_solver_{datetime.now().strftime('%Y%m%d')}.log"
            
            logging.basicConfig(
                level=logging.INFO,
                format=log_format,
                handlers=[
                    logging.FileHandler(log_filename, encoding='utf-8'),
                    logging.StreamHandler()
                ]
            )
            
        except Exception as e:
            print(f"خطأ في إعداد السجلات: {e}")
    
    def _load_session_data(self):
        """تحميل بيانات الجلسة المحفوظة"""
        try:
            session_file = self.config_manager.get_secure_path('PATHS', 'memory_dir', 'session_data.json')
            
            if session_file.exists():
                with open(session_file, 'r', encoding='utf-8') as f:
                    saved_session = json.load(f)
                    
                self.current_session.update({
                    'total_captchas_solved': saved_session.get('total_captchas_solved', 0),
                    'total_cycles': saved_session.get('total_cycles', 0),
                    'best_solving_time': saved_session.get('best_solving_time', float('inf')),
                    'success_rate_history': saved_session.get('success_rate_history', [])
                })
                
                logging.info("تم تحميل بيانات الجلسة السابقة")
                
        except Exception as e:
            logging.warning(f"خطأ في تحميل بيانات الجلسة: {e}")
    
    def _save_session_data(self):
        """حفظ بيانات الجلسة"""
        try:
            session_file = self.config_manager.get_secure_path('PATHS', 'memory_dir', 'session_data.json')
            
            session_data = {
                'last_save': datetime.now().isoformat(),
                'total_captchas_solved': self.current_session.get('total_captchas_solved', 0),
                'total_cycles': self.current_session.get('total_cycles', 0),
                'best_solving_time': self.current_session.get('best_solving_time', float('inf')),
                'success_rate_history': self.current_session.get('success_rate_history', []),
                'current_session_stats': {
                    'cycles_completed': self.current_session['cycles_completed'],
                    'captchas_solved': self.current_session['captchas_solved'],
                    'errors_encountered': self.current_session['errors_encountered'],
                    'session_duration': time.time() - self.current_session['start_time']
                }
            }
            
            with open(session_file, 'w', encoding='utf-8') as f:
                json.dump(session_data, f, indent=2, ensure_ascii=False)
                
        except Exception as e:
            logging.warning(f"خطأ في حفظ بيانات الجلسة: {e}")
    
    async def solve_captcha_cycle(self) -> Dict[str, any]:
        """دورة حل كابتشا واحدة مع التعلم التكيفي"""
        cycle_start_time = time.time()
        cycle_result = {
            'success': False,
            'steps_completed': [],
            'errors': [],
            'total_time': 0,
            'retry_count': 0,
            'learning_applied': False
        }
        
        try:
            logging.info("🔄 بدء دورة حل كابتشا جديدة مع التعلم التكيفي")
            
            # فحص الموارد
            resources = self.performance_monitor.check_resources()
            if not resources['memory_ok'] or not resources['cpu_ok']:
                logging.warning(f"⚠️ موارد النظام محدودة: ذاكرة={resources['memory_mb']:.1f}MB، معالج={resources['cpu_percent']:.1f}%")
                
                if self.performance_monitor.should_cleanup():
                    self.performance_monitor.cleanup_memory()
                    self.screenshot_manager.cleanup_cache()
            
            # الخطوة 1: البحث عن reCAPTCHA مع التعلم
            recaptcha_result = await self._find_and_click_recaptcha_adaptive()
            cycle_result['steps_completed'].append('recaptcha_search')
            
            if not recaptcha_result['success']:
                if recaptcha_result.get('not_found'):
                    cycle_result['success'] = True
                    cycle_result['reason'] = 'no_captcha_found'
                    return cycle_result
                else:
                    cycle_result['errors'].append(recaptcha_result.get('error', 'unknown_recaptcha_error'))
                    return cycle_result
            
            cycle_result['learning_applied'] = recaptcha_result.get('learning_applied', False)
            
            # الخطوة 2: انتظار Verified مع التعلم
            verified_result = await self._wait_and_click_verified_adaptive()
            cycle_result['steps_completed'].append('verified_wait')
            
            if not verified_result['success']:
                if cycle_result['retry_count'] < self.max_retry_attempts:
                    cycle_result['retry_count'] += 1
                    logging.info(f"🔄 إعادة المحاولة {cycle_result['retry_count']}/{self.max_retry_attempts}")
                    
                    await asyncio.sleep(self.retry_delay_multiplier * cycle_result['retry_count'])
                    return await self.solve_captcha_cycle()
                else:
                    cycle_result['errors'].append(verified_result.get('error', 'verified_timeout'))
                    return cycle_result
            
            # الخطوة 3: انتظار النجاح النهائي مع التعلم
            success_result = await self._wait_for_final_success_adaptive()
            cycle_result['steps_completed'].append('success_wait')
            
            if success_result['success']:
                cycle_result['success'] = True
                cycle_result['success_element'] = success_result.get('element')
                
                # تحديث الإحصائيات
                self.current_session['captchas_solved'] += 1
                solving_time = time.time() - cycle_start_time
                
                if solving_time < self.current_session.get('best_solving_time', float('inf')):
                    self.current_session['best_solving_time'] = solving_time
                
                logging.info(f"🎉 تم حل الكابتشا بنجاح في {solving_time:.2f} ثانية مع التعلم التكيفي")
                
                # تدريب النماذج على النجاح
                await self._update_ml_with_success(cycle_result)
                
            else:
                cycle_result['errors'].append(success_result.get('error', 'final_success_timeout'))
            
        except Exception as e:
            logging.error(f"❌ خطأ في دورة حل الكابتشا: {e}")
            cycle_result['errors'].append(f"cycle_exception: {str(e)}")
            self.current_session['errors_encountered'] += 1
        
        finally:
            cycle_result['total_time'] = time.time() - cycle_start_time
            self.current_session['cycles_completed'] += 1
            self.current_session['last_activity'] = time.time()
            
            # حفظ تلقائي للتعلم
            self.learning_engine.auto_save_if_needed()
            
            # حفظ تلقائي للجلسة
            if self._should_auto_save_session():
                self._save_session_data()
                self.current_session['last_auto_save'] = time.time()
        
        return cycle_result
    
    async def _find_and_click_recaptcha_adaptive(self) -> Dict[str, any]:
        """البحث والنقر على reCAPTCHA مع التعلم التكيفي"""
        try:
            screenshot = self.screenshot_manager.capture_smart_screenshot()
            if screenshot is None:
                return {'success': False, 'error': 'screenshot_failed'}
            
            # استخدام العتبة المثلى من التعلم
            optimal_threshold = self.learning_engine.get_optimal_confidence_threshold('recaptcha')
            
            detection = self.template_matcher.find_template_advanced('recaptcha', screenshot, threshold=optimal_threshold)
            
            if not detection:
                return {'success': False, 'not_found': True}
            
            # استخراج الميزات للتعلم
            region = self._extract_detection_region(screenshot, detection)
            features = self.ml_engine.feature_extractor.extract_comprehensive_features(region)
            
            # كشف الشذوذ
            anomaly_result = self.ml_engine.detect_anomaly(features)
            if anomaly_result['is_anomaly']:
                logging.warning(f"⚠️ تم كشف شذوذ في reCAPTCHA: {anomaly_result['anomaly_score']:.3f}")
            
            # النقر الذكي مع التعلم
            click_success = self.click_manager.smart_click(detection, 'recaptcha')
            
            if click_success:
                return {
                    'success': True,
                    'detection': detection,
                    'features': features,
                    'anomaly_detected': anomaly_result['is_anomaly'],
                    'learning_applied': True,
                    'threshold_used': optimal_threshold
                }
            else:
                return {'success': False, 'error': 'click_failed'}
                
        except Exception as e:
            logging.error(f"خطأ في البحث عن reCAPTCHA: {e}")
            return {'success': False, 'error': f'exception: {str(e)}'}
    
    async def _wait_and_click_verified_adaptive(self) -> Dict[str, any]:
        """انتظار والنقر على Verified مع التعلم التكيفي"""
        try:
            # انتظار ظهور Verified مع التعلم
            wait_result = self.wait_manager.smart_wait_for_element(['verified'], context='verified')
            
            if not wait_result['success']:
                return {'success': False, 'error': 'verified_not_found', 'wait_result': wait_result}
            
            # النقر على Verified مع التعلم
            click_success = self.click_manager.smart_click(wait_result['detection'], 'verified')
            
            if click_success:
                return {
                    'success': True,
                    'detection': wait_result['detection'],
                    'wait_time': wait_result['elapsed_time'],
                    'learning_applied': True
                }
            else:
                return {'success': False, 'error': 'verified_click_failed'}
                
        except Exception as e:
            logging.error(f"خطأ في انتظار Verified: {e}")
            return {'success': False, 'error': f'exception: {str(e)}'}
    
    async def _wait_for_final_success_adaptive(self) -> Dict[str, any]:
        """انتظار النجاح النهائي مع التعلم التكيفي"""
        try:
            # انتظار ظهور Done أو Google مع التعلم
            wait_result = self.wait_manager.smart_wait_for_element(['done', 'google'], context='success')
            
            if wait_result['success']:
                return {
                    'success': True,
                    'element': wait_result['element'],
                    'detection': wait_result['detection'],
                    'wait_time': wait_result['elapsed_time'],
                    'learning_applied': True
                }
            else:
                return {'success': False, 'error': 'final_success_timeout', 'wait_result': wait_result}
                
        except Exception as e:
            logging.error(f"خطأ في انتظار النجاح النهائي: {e}")
            return {'success': False, 'error': f'exception: {str(e)}'}
    
    def _extract_detection_region(self, image: np.ndarray, detection: Dict) -> np.ndarray:
        """استخراج منطقة الكشف من الصورة"""
        try:
            x, y = detection['location']
            w, h = detection['template_size']
            
            x = max(0, x)
            y = max(0, y)
            w = min(w, image.shape[1] - x)
            h = min(h, image.shape[0] - y)
            
            return image[y:y+h, x:x+w]
            
        except Exception as e:
            logging.warning(f"خطأ في استخراج منطقة الكشف: {e}")
            return image[:50, :100]
    
    async def _update_ml_with_success(self, cycle_result: Dict):
        """تحديث نماذج ML بناءً على النجاح"""
        try:
            if not cycle_result.get('success'):
                return
            
            training_sample = {
                'result': 'success',
                'confidence': 0.9,
                'duration': cycle_result['total_time'],
                'steps': cycle_result['steps_completed'],
                'advanced_features': {},
                'timestamp': datetime.now().isoformat(),
                'learning_applied': cycle_result.get('learning_applied', False)
            }
            
            if 'features' in cycle_result:
                training_sample['advanced_features'] = cycle_result['features']
            
            asyncio.create_task(self._async_model_training([training_sample]))
            
        except Exception as e:
            logging.warning(f"خطأ في تحديث ML: {e}")
    
    async def _async_model_training(self, training_samples: List[Dict]):
        """تدريب النماذج بشكل غير متزامن"""
        try:
            loop = asyncio.get_event_loop()
            with ThreadPoolExecutor(max_workers=1) as executor:
                await loop.run_in_executor(
                    executor, 
                    self.ml_engine.train_models, 
                    training_samples
                )
        except Exception as e:
            logging.warning(f"خطأ في التدريب غير المتزامن: {e}")
    
        def _should_auto_save_session(self) -> bool:
          """تحديد ما إذا كان يجب الحفظ التلقائي للجلسة"""
        try:
            auto_save_interval = int(self.config_manager.config.get('LEARNING', 'auto_save_interval', fallback='300'))
            time_since_last_save = time.time() - self.current_session.get('last_auto_save', 0)
            return time_since_last_save > auto_save_interval
        except:
            return False
    
    async def run_continuous_monitoring(self):
        """تشغيل المراقبة المستمرة مع التعلم التكيفي"""
        logging.info("🚀 بدء المراقبة المستمرة مع التعلم التكيفي المتقدم")
        
        last_stats_print = time.time()
        last_save = time.time()
        consecutive_errors = 0
        max_consecutive_errors = 10
        
        try:
            while True:
                cycle_start = time.time()
                
                try:
                    # تنفيذ دورة حل مع التعلم
                    cycle_result = await self.solve_captcha_cycle()
                    
                    # تحليل النتيجة
                    if cycle_result['success']:
                        consecutive_errors = 0
                        self.system_state = "operating_normally"
                        
                        # تحديث معدل النجاح
                        if 'success_rate_history' not in self.current_session:
                            self.current_session['success_rate_history'] = []
                        
                        current_success_rate = self.current_session['captchas_solved'] / max(self.current_session['cycles_completed'], 1)
                        self.current_session['success_rate_history'].append({
                            'timestamp': time.time(),
                            'success_rate': current_success_rate,
                            'learning_applied': cycle_result.get('learning_applied', False)
                        })
                        
                        # الاحتفاظ بآخر 100 نقطة فقط
                        if len(self.current_session['success_rate_history']) > 100:
                            self.current_session['success_rate_history'] = self.current_session['success_rate_history'][-100:]
                        
                    else:
                        consecutive_errors += 1
                        
                        if consecutive_errors >= max_consecutive_errors:
                            logging.error(f"❌ تم الوصول للحد الأقصى من الأخطاء المتتالية: {consecutive_errors}")
                            self.system_state = "error_recovery"
                            
                            await self._attempt_system_recovery()
                            consecutive_errors = 0
                    
                    # طباعة الإحصائيات مع معلومات التعلم
                    if time.time() - last_stats_print > 300:  # كل 5 دقائق
                        await self._print_comprehensive_stats_with_learning()
                        last_stats_print = time.time()
                    
                    # حفظ البيانات
                    if time.time() - last_save > 600:  # كل 10 دقائق
                        self._save_session_data()
                        self.learning_engine._save_learning_data()
                        last_save = time.time()
                        logging.info("💾 تم الحفظ التلقائي الشامل")
                    
                    # تنظيف الذاكرة إذا لزم الأمر
                    if self.performance_monitor.should_cleanup():
                        self.performance_monitor.cleanup_memory()
                        self.screenshot_manager.cleanup_cache()
                        self.template_matcher._cleanup_cache()
                    
                    # تحديد سرعة الدورة التالية بناءً على التعلم
                    cycle_duration = time.time() - cycle_start
                    
                    if cycle_result['success']:
                        # دورة ناجحة، انتظار قصير مع تحسين تدريجي
                        optimal_delay = self.learning_engine.get_optimal_click_delay()
                        await asyncio.sleep(max(0.3, optimal_delay * 0.5))
                    elif cycle_result.get('reason') == 'no_captcha_found':
                        # لا توجد كابتشا، انتظار عادي
                        await asyncio.sleep(1.0)
                    else:
                        # خطأ، انتظار أطول مع تعلم من الفشل
                        failure_delay = self.learning_engine.get_optimal_click_delay() * 2
                        await asyncio.sleep(min(5.0, failure_delay))
                
                except KeyboardInterrupt:
                    logging.info("🛑 تم إيقاف النظام بواسطة المستخدم")
                    break
                    
                except Exception as e:
                    logging.error(f"❌ خطأ في دورة المراقبة: {e}")
                    consecutive_errors += 1
                    
                    if consecutive_errors >= max_consecutive_errors:
                        logging.critical(f"💥 خطأ حرج: {consecutive_errors} أخطاء متتالية")
                        await self._attempt_system_recovery()
                        consecutive_errors = 0
                    
                    await asyncio.sleep(5.0)
        
        except Exception as e:
            logging.critical(f"💥 خطأ حرج في النظام: {e}")
            
        finally:
            # حفظ البيانات النهائية
            self._save_session_data()
            self.learning_engine._save_learning_data()
            await self._print_final_summary_with_learning()
            logging.info("🏁 تم إنهاء المراقبة المستمرة مع حفظ جميع بيانات التعلم")
    
    async def _attempt_system_recovery(self):
        """محاولة استعادة النظام من الأخطاء"""
        try:
            logging.info("🔧 بدء محاولة استعادة النظام...")
            
            # تنظيف شامل للذاكرة
            self.performance_monitor.cleanup_memory()
            self.screenshot_manager.cleanup_cache()
            self.template_matcher._cleanup_cache()
            
            # إعادة تحميل القوالب
            self.template_matcher._load_templates()
            
            # حفظ بيانات التعلم قبل الاستعادة
            self.learning_engine._save_learning_data()
            
            # إعادة تعيين الحالات
            self.system_state = "recovering"
            
            await asyncio.sleep(10.0)
            
            # اختبار بسيط للنظام
            test_screenshot = self.screenshot_manager.capture_smart_screenshot(force_full=True)
            
            if test_screenshot is not None:
                logging.info("✅ تم استعادة النظام بنجاح مع الاحتفاظ بالتعلم")
                self.system_state = "operating_normally"
                return True
            else:
                logging.error("❌ فشل في استعادة النظام")
                return False
                
        except Exception as e:
            logging.error(f"❌ خطأ في استعادة النظام: {e}")
            return False
    
    async def _print_comprehensive_stats_with_learning(self):
        """طباعة إحصائيات شاملة مع معلومات التعلم"""
        try:
            session_duration = time.time() - self.current_session['start_time']
            
            print("\n" + "="*80)
            print("📊 إحصائيات النظام المتقدم مع التعلم التكيفي")
            print("="*80)
            
            # إحصائيات الجلسة الحالية
            print(f"⏰ مدة الجلسة: {session_duration/3600:.1f} ساعة")
            print(f"🔄 دورات مكتملة: {self.current_session['cycles_completed']:,}")
            print(f"🎯 كابتشا محلولة: {self.current_session['captchas_solved']}")
            print(f"❌ أخطاء واجهت: {self.current_session['errors_encountered']}")
            
            if self.current_session['cycles_completed'] > 0:
                success_rate = self.current_session['captchas_solved'] / self.current_session['cycles_completed']
                print(f"📈 معدل النجاح: {success_rate:.1%}")
            
            if self.current_session.get('best_solving_time', float('inf')) != float('inf'):
                print(f"⚡ أفضل وقت حل: {self.current_session['best_solving_time']:.2f}s")
            
            # إحصائيات التعلم التكيفي
            learning_stats = self.learning_engine.get_learning_statistics()
            print(f"\n🧠 إحصائيات التعلم التكيفي:")
            print(f"   📚 عينات التعلم: {learning_stats['total_samples']}")
            print(f"   ✅ تكيفات ناجحة: {learning_stats['successful_adaptations']}")
            print(f"   ❌ تكيفات فاشلة: {learning_stats['failed_adaptations']}")
            print(f"   📊 معدل التكيف: {learning_stats['adaptation_rate']:.1%}")
            
            # المعاملات المحسنة
            current_params = learning_stats['current_parameters']
            print(f"\n⚙️ المعاملات المحسنة:")
            print(f"   ⏳ مهلة Verified: {current_params['verified_timeout']:.1f}s")
            print(f"   🎯 مهلة النجاح: {current_params['success_timeout']:.1f}s")
            print(f"   🖱️ تأخير النقر: {current_params['click_delay']:.3f}s")
            print(f"   🎯 عتبة الثقة: {current_params['confidence_threshold']:.3f}") 
            print(f"⏰ إجمالي مدة التشغيل: {total_duration/3600:.2f} ساعة")
            print(f"🔄 إجمالي الدورات: {self.current_session['cycles_completed']:,}")
            print(f"🎯 إجمالي الكابتشا المحلولة: {self.current_session['captchas_solved']}")
            print(f"❌ إجمالي الأخطاء: {self.current_session['errors_encountered']}")
            
            if self.current_session['cycles_completed'] > 0:
                overall_success_rate = self.current_session['captchas_solved'] / self.current_session['cycles_completed']
                print(f"📈 معدل النجاح الإجمالي: {overall_success_rate:.1%}")
                
                avg_cycle_time = total_duration / self.current_session['cycles_completed']
                print(f"⚡ متوسط وقت الدورة: {avg_cycle_time:.2f}s")
            
            if self.current_session.get('best_solving_time', float('inf')) != float('inf'):
                print(f"🏆 أفضل وقت حل: {self.current_session['best_solving_time']:.2f}s")
            
            # إنجازات التعلم التكيفي
            learning_stats = self.learning_engine.get_learning_statistics()
            print(f"\n🧠 إنجازات التعلم التكيفي:")
            print(f"   📚 إجمالي عينات التعلم: {learning_stats['total_samples']}")
            print(f"   ✅ تكيفات ناجحة: {learning_stats['successful_adaptations']}")
            print(f"   📊 معدل التكيف النهائي: {learning_stats['adaptation_rate']:.1%}")
            
            # تحسينات الأداء المكتسبة
            current_params = learning_stats['current_parameters']
            print(f"\n⚡ تحسينات الأداء المكتسبة:")
            print(f"   ⏳ مهلة Verified محسنة: {current_params['verified_timeout']:.1f}s")
            print(f"   🎯 مهلة النجاح محسنة: {current_params['success_timeout']:.1f}s")
            print(f"   🖱️ تأخير النقر محسن: {current_params['click_delay']:.3f}s")
            print(f"   🎯 عتبة الثقة محسنة: {current_params['confidence_threshold']:.3f}")
            
            # إحصائيات التعلم الآلي
            if self.ml_engine.is_trained:
                print(f"\n🤖 إنجازات التعلم الآلي:")
                print(f"   📚 عينات مدربة: {self.ml_engine.model_stats['training_samples']}")
                print(f"   🎯 نماذج مدربة: {len([m for m in self.ml_engine.models.values() if m is not None])}")
                
                if 'accuracy_scores' in self.ml_engine.model_stats:
                    avg_accuracy = np.mean(list(self.ml_engine.model_stats['accuracy_scores'].values()))
                    print(f"   📊 متوسط دقة النماذج: {avg_accuracy:.1%}")
                
                if HAS_STATSMODELS:
                    print(f"   📈 إحصائيات متقدمة مع statsmodels: متاحة")
            
            # تحليل تطور الأداء
            if 'success_rate_history' in self.current_session and len(self.current_session['success_rate_history']) > 5:
                history = self.current_session['success_rate_history']
                initial_rate = np.mean([h['success_rate'] for h in history[:5]])
                final_rate = np.mean([h['success_rate'] for h in history[-5:]])
                improvement = final_rate - initial_rate
                
                print(f"\n📈 تطور الأداء:")
                print(f"   📊 معدل النجاح الأولي: {initial_rate:.1%}")
                print(f"   📊 معدل النجاح النهائي: {final_rate:.1%}")
                print(f"   {'📈' if improvement > 0 else '📉'} التحسن: {improvement:+.1%}")
            
            # توصيات مخصصة بناءً على التعلم
            print(f"\n💡 توصيات مخصصة بناءً على التعلم:")
            
            if learning_stats['adaptation_rate'] > 0.8:
                print("   🎯 النظام يتعلم بكفاءة عالية - استمر في التشغيل")
            elif learning_stats['adaptation_rate'] < 0.5:
                print("   📚 النظام يحتاج المزيد من البيانات للتعلم الأمثل")
            
            if overall_success_rate > 0.9:
                print("   ⚡ أداء ممتاز - يمكن تقليل أوقات الانتظار أكثر")
            elif overall_success_rate < 0.7:
                print("   🔧 يُنصح بزيادة أوقات الانتظار للحصول على استقرار أكثر")
            
            if self.ml_engine.is_trained:
                print("   🤖 نماذج ML مدربة - ستحصل على تنبؤات أفضل")
            else:
                print("   📊 جمع المزيد من البيانات سيحسن دقة التنبؤات")
            
            # معلومات الحفظ والاستئناف
            print(f"\n💾 معلومات الحفظ:")
            print(f"   📁 بيانات الجلسة: محفوظة")
            print(f"   🧠 بيانات التعلم التكيفي: محفوظة")
            print(f"   🤖 نماذج ML: محفوظة")
            print(f"   📊 إحصائيات الأداء: محفوظة")
            
            print(f"\n🔄 استئناف التشغيل:")
            print(f"   ✅ يمكن استئناف التشغيل مع الاحتفاظ بجميع التحسينات")
            print(f"   🧠 سيتم تطبيق جميع المعاملات المحسنة تلقائياً")
            print(f"   📈 الأداء سيكون أفضل من البداية بفضل التعلم المكتسب")
            
            print(f"\n📊 ملفات النظام:")
            memory_dir = self.config_manager.get_secure_path('PATHS', 'memory_dir', '')
            print(f"   💾 مجلد البيانات: {memory_dir}")
            print(f"   🔧 ملف الإعدادات: {self.config_manager.config_path}")
            
            print("🏁" + "="*78 + "🏁")
            
        except Exception as e:
            logging.error(f"خطأ في طباعة الملخص النهائي: {e}")
    
    def export_comprehensive_report_with_learning(self) -> str:
        """تصدير تقرير شامل مع معلومات التعلم"""
        try:
            report_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            report_file = self.config_manager.get_secure_path(
                'PATHS', 'memory_dir', f'comprehensive_report_with_learning_{report_timestamp}.json'
            )
            
            # جمع جميع الإحصائيات مع التعلم
            learning_stats = self.learning_engine.get_learning_statistics()
            
            report_data = {
                'report_metadata': {
                    'generated_at': datetime.now().isoformat(),
                    'system_version': 'AdvancedCaptchaSolver_v4.0_with_Adaptive_Learning',
                    'report_type': 'comprehensive_system_analysis_with_learning',
                    'statsmodels_available': HAS_STATSMODELS
                },
                'session_summary': self.current_session.copy(),
                'adaptive_learning_stats': learning_stats,
                'performance_metrics': self.performance_monitor.check_resources(),
                'ml_statistics': self.ml_engine.model_stats.copy() if self.ml_engine.is_trained else {},
                'click_analytics': self.click_manager.get_click_statistics(),
                'wait_analytics': self.wait_manager.get_wait_statistics(),
                'system_configuration': {
                    'security_config': asdict(self.security_config),
                    'performance_config': asdict(self.performance_config),
                    'learning_config': asdict(self.learning_config),
                    'solving_strategy': self.solving_strategy,
                    'max_retry_attempts': self.max_retry_attempts
                },
                'learned_optimizations': {
                    'optimal_timeouts': {
                        'verified': learning_stats['current_parameters']['verified_timeout'],
                        'success': learning_stats['current_parameters']['success_timeout']
                    },
                    'optimal_delays': {
                        'click_delay': learning_stats['current_parameters']['click_delay']
                    },
                    'optimal_thresholds': {
                        'confidence_threshold': learning_stats['current_parameters']['confidence_threshold']
                    }
                },
                'recommendations': self._generate_system_recommendations_with_learning()
            }
            
            # حفظ التقرير
            with open(report_file, 'w', encoding='utf-8') as f:
                json.dump(report_data, f, indent=2, ensure_ascii=False, default=str)
            
            logging.info(f"📊 تم تصدير التقرير الشامل مع التعلم إلى: {report_file}")
            return str(report_file)
            
        except Exception as e:
            logging.error(f"خطأ في تصدير التقرير: {e}")
            return ""
    
    def _generate_system_recommendations_with_learning(self) -> List[str]:
        """توليد توصيات تحسين النظام مع التعلم"""
        recommendations = []
        
        try:
            # تحليل معدل النجاح
            if self.current_session['cycles_completed'] > 0:
                success_rate = self.current_session['captchas_solved'] / self.current_session['cycles_completed']
                
                if success_rate < 0.5:
                    recommendations.append("معدل النجاح منخفض - النظام يتعلم، استمر في التشغيل لتحسين الأداء")
                elif success_rate > 0.9:
                    recommendations.append("معدل نجاح ممتاز مع التعلم التكيفي - النظام محسن بشكل مثالي")
            
            # تحليل التعلم التكيفي
            learning_stats = self.learning_engine.get_learning_statistics()
            
            if learning_stats['adaptation_rate'] > 0.8:
                recommendations.append("معدل تكيف ممتاز - النظام يتعلم بكفاءة عالية")
            elif learning_stats['adaptation_rate'] < 0.5:
                recommendations.append("معدل تكيف منخفض - النظام يحتاج المزيد من الوقت للتعلم")
            
            if learning_stats['total_samples'] < 50:
                recommendations.append("عدد عينات التعلم قليل - استمر في التشغيل لجمع المزيد من البيانات")
            elif learning_stats['total_samples'] > 500:
                recommendations.append("قاعدة بيانات تعلم ممتازة - النظام قادر على التنبؤ بدقة عالية")
            
            # تحليل الأداء
            resources = self.performance_monitor.check_resources()
            if not resources['memory_ok']:
                recommendations.append("استخدام الذاكرة مرتفع - قم بتقليل حجم البافر أو تفعيل التنظيف التلقائي")
            
            # تحليل ML
            if not self.ml_engine.is_trained:
                recommendations.append("نماذج ML غير مدربة - التعلم التكيفي سيحسن الأداء تدريجياً")
            elif self.ml_engine.model_stats.get('training_samples', 0) > 100:
                recommendations.append("نماذج ML مدربة جيداً - ستحصل على تنبؤات دقيقة")
            
            # تحليل تطور الأداء
            if 'success_rate_history' in self.current_session and len(self.current_session['success_rate_history']) > 5:
                history = self.current_session['success_rate_history']
                recent_improvement = history[-1]['success_rate'] - history[-5]['success_rate']
                
                if recent_improvement > 0.1:
                    recommendations.append("الأداء يتحسن بسرعة بفضل التعلم التكيفي")
                elif recent_improvement < -0.1:
                    recommendations.append("تراجع في الأداء - قد تحتاج لمراجعة الإعدادات")
            
            # توصيات خاصة بـ statsmodels
            if HAS_STATSMODELS:
                recommendations.append("statsmodels متاحة - ستحصل على إحصائيات متقدمة وفترات ثقة دقيقة")
            else:
                recommendations.append("تثبيت statsmodels سيوفر إحصائيات أكثر تقدماً")
            
            # توصيات عامة
            if len(recommendations) == 0:
                recommendations.append("النظام يعمل بكفاءة عالية مع التعلم التكيفي المتقدم!")
            
            return recommendations
            
        except Exception as e:
            logging.error(f"خطأ في توليد التوصيات: {e}")
            return ["خطأ في تحليل النظام - راجع السجلات للتفاصيل"]

# ================================
# Main Execution
# ================================

async def main():
    """الدالة الرئيسية للتشغيل مع التعلم التكيفي"""
    print("🚀" + "="*78 + "🚀")
    print("🤖 نظام حل الكابتشا المتقدم مع التعلم التكيفي - الإصدار 4.0")
    print("🚀" + "="*78 + "🚀")
    print()
    print("✨ المميزات المتقدمة مع التعلم التكيفي:")
    print("   🧠 تعلم آلي متقدم مع Random Forest و Neural Networks")
    print("   🎯 تعلم تكيفي للأوقات المثلى والمعاملات")
    print("   🖱️ نقر ذكي مع تعلم الأنماط الناجحة")
    print("   ⏳ انتظار تكيفي مع تحسين مستمر للسرعة")
    print("   🔒 طبقات أمان متعددة مع تشفير البيانات")
    print("   ⚡ إدارة ذاكرة ذكية مع تنظيف تلقائي")
    print("   🎯 كشف متقدم مع مقاومة التغييرات")
    print("   📊 مراقبة أداء شاملة مع إحصائيات مفصلة")
    print("   🔄 إعادة محاولة ذكية مع استعادة تلقائية")
    print("   💾 حفظ تلقائي مع استئناف التعلم")
    print("   📈 تتبع تطور الأداء والتحسين المستمر")
    print("   🛡️ معالجة شاملة للاستثناءات")
    
    if HAS_STATSMODELS:
        print("   📊 إحصائيات متقدمة مع statsmodels")
    else:
        print("   ⚠️ statsmodels غير مثبتة - سيتم استخدام إحصائيات بسيطة")
    
    print()
    print("🔧 إعدادات النظام المحسنة:")
    print("   📁 الملفات: config/, templates/, logs/, captcha_memory/")
    print("   🔐 الأمان: تشفير البيانات + مسارات آمنة")
    print("   💾 الذاكرة: تنظيف تلقائي + مراقبة الاستخدام")
    print("   🚀 الأداء: معالجة متوازية + كاش ذكي")
    print("   🧠 التعلم: حفظ تلقائي كل 5 دقائق")
    print()
    print("⚠️ متطلبات التشغيل:")
    print("   • Python 3.8+ مع المكتبات المطلوبة")
    print("   • ملفات القوالب في مجلد templates/")
    print("   • صلاحيات الوصول للشاشة والماوس")
    print("   • مساحة كافية لحفظ البيانات والتعلم")
    print("   • اتصال إنترنت لتحديث المكتبات (اختياري)")
    print()
    print("🎮 التحكم:")
    print("   • Ctrl+C للإيقاف الآمن مع حفظ التعلم")
    print("   • سيتم حفظ جميع البيانات والتحسينات تلقائياً")
    print("   • يمكن استئناف التشغيل مع الاستفادة من التعلم السابق")
    print("   • النظام سيتحسن تلقائياً مع كل استخدام")
    print()
    print("🚀" + "="*78 + "🚀")
    
    try:
        # إنشاء النظام المتقدم مع التعلم
        solver = AdvancedCaptchaSolver()
        
        # عرض حالة النظام
        print("\n📋 حالة النظام:")
        print(f"   🤖 نماذج ML: {'✅ مدربة' if solver.ml_engine.is_trained else '❌ غير مدربة'}")
        print(f"   🖼️ القوالب: {len(solver.template_matcher.templates)} قالب محمل")
        print(f"   🧠 التعلم التكيفي: ✅ نشط")
        print(f"   🔧 الحالة: {solver.system_state}")
        
        # عرض المعاملات المحسنة
        learning_stats = solver.learning_engine.get_learning_statistics()
        if learning_stats['total_samples'] > 0:
            print(f"   📚 عينات التعلم: {learning_stats['total_samples']}")
            print(f"   📊 معدل التكيف: {learning_stats['adaptation_rate']:.1%}")
            
            current_params = learning_stats['current_parameters']
            print(f"   ⏳ مهلة Verified محسنة: {current_params['verified_timeout']:.1f}s")
            print(f"   🎯 مهلة النجاح محسنة: {current_params['success_timeout']:.1f}s")
            print(f"   🖱️ تأخير النقر محسن: {current_params['click_delay']:.3f}s")
        else:
            print(f"   📚 عينات التعلم: 0 (سيبدأ التعلم مع الاستخدام)")
        
        # فحص الموارد
        resources = solver.performance_monitor.check_resources()
        print(f"   🧠 الذاكرة: {resources['memory_mb']:.1f}MB")
        print(f"   ⚙️ المعالج: {resources['cpu_percent']:.1f}%")
        
        # فحص statsmodels
        if HAS_STATSMODELS:
            print(f"   📊 statsmodels: ✅ متاحة للإحصائيات المتقدمة")
        else:
            print(f"   📊 statsmodels: ❌ غير متاحة")
        
        print("\n🚀 بدء المراقبة المستمرة مع التعلم التكيفي...")
        print("💡 النظام سيتعلم ويتحسن تلقائياً مع كل استخدام")
        print("📊 ستظهر الإحصائيات مع معلومات التعلم كل 5 دقائق")
        print("💾 سيتم الحفظ التلقائي للتعلم كل 5 دقائق")
        print("🎯 الأداء سيتحسن تدريجياً مع جمع المزيد من البيانات")
        print("⚡ أوقات الانتظار والنقر ستصبح أكثر دقة مع التعلم")
        print()
        
        # بدء المراقبة المستمرة مع التعلم
        await solver.run_continuous_monitoring()
        
    except KeyboardInterrupt:
        print("\n🛑 تم إيقاف النظام بواسطة المستخدم")
        
    except Exception as e:
        print(f"\n💥 خطأ حرج في النظام: {e}")
        logging.critical(f"خطأ حرج في main: {e}")
        
    finally:
        try:
            # تصدير تقرير نهائي مع التعلم
            if 'solver' in locals():
                report_file = solver.export_comprehensive_report_with_learning()
                if report_file:
                    print(f"📊 تم تصدير التقرير النهائي مع التعلم: {report_file}")
                
                print("\n💾 تم حفظ جميع البيانات والتعلم المكتسب:")
                print("   🧠 بيانات التعلم التكيفي")
                print("   🤖 نماذج التعلم الآلي")
                print("   📊 إحصائيات الأداء")
                print("   ⚙️ المعاملات المحسنة")
                print("   📈 تاريخ التحسينات")
                
                print("\n🔄 عند إعادة التشغيل:")
                print("   ✅ سيتم تطبيق جميع التحسينات المكتسبة")
                print("   ⚡ الأداء سيكون أفضل من البداية")
                print("   🧠 التعلم سيستمر من حيث توقف")
                print("   📈 معدل النجاح سيكون أعلى")
                
        except Exception as e:
            print(f"⚠️ خطأ في الحفظ النهائي: {e}")
        
        print("\n🏁 تم إنهاء النظام بأمان مع حفظ جميع التحسينات")

# ================================
# Entry Point
# ================================

if __name__ == "__main__":
    try:
        # فحص المتطلبات الأساسية
        required_modules = [
            'mss', 'cv2', 'numpy', 'pyautogui', 'sklearn', 
            'skimage', 'scipy', 'cryptography', 'psutil'
        ]
        
        missing_modules = []
        for module in required_modules:
            try:
                __import__(module)
            except ImportError:
                missing_modules.append(module)
        
        if missing_modules:
            print("❌ مكتبات مفقودة:")
            for module in missing_modules:
                print(f"   • {module}")
            print("\n📦 قم بتثبيت المكتبات المفقودة:")
            print("pip install mss opencv-python numpy pyautogui scikit-learn")
            print("pip install scikit-image scipy cryptography psutil")
            print("pip install statsmodels  # للإحصائيات المتقدمة")
            exit(1)
        
        # فحص إصدار Python
        import sys
        if sys.version_info < (3, 8):
            print("❌ يتطلب Python 3.8 أو أحدث")
            exit(1)
        
        # إنشاء المجلدات المطلوبة
        required_dirs = ['config', 'templates', 'logs', 'captcha_memory', 'temp']
        for dir_name in required_dirs:
            Path(dir_name).mkdir(exist_ok=True)
        
        # فحص ملفات القوالب
        template_files = ['recaptcha.png', 'verified.png', 'done.png', 'google.png']
        missing_templates = []
        
        for template in template_files:
            template_path = Path('templates') / template
            if not template_path.exists():
                missing_templates.append(template)
        
        if missing_templates:
            print("⚠️ ملفات قوالب مفقودة:")
            for template in missing_templates:
                print(f"   • templates/{template}")
            print("\n💡 سيتم إنشاء قوالب احتياطية تلقائياً")
            print("🔧 للحصول على أفضل أداء، ضع ملفات القوالب الصحيحة في مجلد templates/")
        
        # فحص statsmodels
        try:
            import statsmodels
            print("✅ statsmodels متاحة - ستحصل على إحصائيات متقدمة")
        except ImportError:
            print("⚠️ statsmodels غير مثبتة - سيتم استخدام إحصائيات بسيطة")
            print("💡 لتثبيت statsmodels: pip install statsmodels")
        
        # تشغيل النظام
        asyncio.run(main())
        
    except KeyboardInterrupt:
        print("\n🛑 تم إلغاء التشغيل")
        
    except Exception as e:
        print(f"\n💥 خطأ في بدء التشغيل: {e}")
        logging.critical(f"خطأ في بدء التشغيل: {e}")
        
    finally:
        print("\n👋 شكراً لاستخدام نظام حل الكابتشا المتقدم مع التعلم التكيفي!")
        print("🧠 كلما استخدمته أكثر، كلما أصبح أذكى وأسرع!")
        print("💾 جميع التحسينات محفوظة للاستخدام القادم")

KeyboardInterrupt: 